In [4]:
# Including Libraries
from datetime import datetime
from elasticsearch import Elasticsearch
from superelasticsearch import SuperElasticsearch
#from  pprint import pprint
import pandas as pd
%run '/home/anshul.jain/code/utilities/imports.ipynb'
import math
import re
es = SuperElasticsearch()
#To run on local
ES_CLIENT = SuperElasticsearch('10.240.0.146:9200', http_auth=('anshul.jain','Ww236wdesd'), timeout=60)
#ES_CLIENT = Elasticsearch('10.240.0.146:9200', http_auth=('anshul.jain','Ww236wdesd'), timeout=60)

# To  run on server
#ES_CLIENT = Elasticsearch('10.240.0.15', timeout=60)
# index_name = 'clinical_trials_alias'
#index_name = 'clinical_trials_alias'
#index_name = 'publication_alias'
import itertools
import datetime
import ast
#r = ast.literal_eval(df['Clinical_trials'][0])
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from give_dataframe import Data_preprocessing
import clinicaltrials as ct
ES_CLIENT
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.utils import class_weight

import pandas as pd
import os
import re
import time
import datetime
import re
import numpy as np

import pickle

In [5]:
os.environ['KERAS_BACKEND'] = 'tensorflow'
from itertools import repeat
from nltk.tokenize import word_tokenize
from keras.layers import Dense, Activation, Input, Dropout, Conv1D
from keras.layers import BatchNormalization, MaxPooling1D, Flatten, concatenate, Embedding
from keras.layers import Reshape, multiply, Lambda
from keras.models import Sequential , Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
from keras import losses, regularizers
from nltk import word_tokenize, pos_tag
from keras import losses
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import GridSearchCV
import gc

In [6]:
# Generates Elasticsearch query
#Version Index
# index_name = 'clinical_trials_finance_alias'
#Latest Version Index
index_name = 'clinical_trials_alias'
def get_query_v2(filters_or,filters_and,filters_boost):
    query = {
                "query": {
                       "bool": {
                          "must":     [],
                           "should" : [],
                           "must_not" : []
                        }
                 }
            }


    for item in filters_or:
        field = item['field']
        values = item['values']
        condition = item.get('condition','or')
#         condition = item.get('exclude','or')
        temp = {"bool": { "should":[],"must":[]}}
        if type(values) == list:
            for term in values:
                if condition.lower() == "and":
                    temp['bool']['must'].append(eval('{"match_phrase":{"%s":"%s"}}' %(item,term)))
                else:
                    temp['bool']['should'].append(eval('{"match_phrase":{"%s":"%s"}}' %(field,term)))
            query['query']['bool']['must'].append(temp)
        else:
            temp['bool']['must'].append(eval('{"match_phrase":{"%s":"%s"}}' %(field,values)))
            query['query']['bool']['must'].append(temp)

    for item in filters_and:
        field = item['field']
        values = item['values']
        condition = item.get('condition','or')
#         condition = item.get('exclude','or')
        temp = {"bool": { "should":[],"must":[]}}
        if type(values) == list:
            for term in values:
                if condition.lower() == "and":
                    temp['bool']['must'].append(eval('{"match_phrase":{"%s":"%s"}}' %(item,term)))
                else:
                    temp['bool']['should'].append(eval('{"match_phrase":{"%s":"%s"}}' %(field,term)))
            query['query']['bool']['must'].append(temp)
        else:
            temp['bool']['must'].append(eval('{"match_phrase":{"%s":"%s"}}' %(field,values)))
            query['query']['bool']['must'].append(temp)

    for item in filters_boost:
        field = item['field']
        values = item['values']
        temp = []
        if type(values) == list:
            for term in values:
                temp.append(eval('{"match_phrase":{"%s":"%s"}}' %(field,term)))
            query['query']['bool']['should'].append(temp)
        else:
            temp.append(eval('{"match_phrase":{"%s":"%s"}}' %(field,values)))
            query['query']['bool']['should'].append(temp)
    #from pprint import pprint 
    #pprint(query)
    return query

# Gives the mapping of fields present in elasticsearch index
def get_fields(alias):
    temp = ES_CLIENT.indices.get_mapping(index=alias)
    indice_name = list(temp.keys())[0]
    mapping = list(temp[indice_name]['mappings'].items())[0][1]['properties']
    return mapping.keys()

# Anshul's version of the below fn
def get_CTlist_from_ES(listofCTs,search_field,field_values):
    listout = []
    for j in range(0,math.ceil(len(listofCTs)/1000)):
        rangex = j*1000
        rangey = (j+1)*1000
        filters_or = [{'field':search_field,
                      'values':field_values}]
        filters_and = [{'field':'clinical_id',
                      'values':listofCTs[rangex:rangey]}]
        query = get_query_v2(filters_or,filters_and,[])
        try:
            a = ES_CLIENT.search(body=query,scroll='10s', request_timeout=60,index=index_name,size=1000)['hits']['hits']
            for i in a:
                listout = listout + [i['_source']['clinical_id']]
        except:
            import traceback 
            ### print(traceback.## print_exc())
            pass
    return list(set(listout))

def get_CTs_from_ES(listofCTs):
    #print (len(listofCTs),math.ceil(len(listofCTs)/1000))
    listout = []
    for j in range(0,math.ceil(len(listofCTs)/1000)):
    #for j in range(0,1):
        rangex = j*1000
        rangey = (j+1)*1000
        filters_or = [{'field': 'clinical_id',
                    'values':listofCTs[rangex:rangey]
                    }]
        '''
        filters_and = [{'field': 'phase',
                    'values':'Phase 3'
                    }]
        '''
        #query = get_query_v2(filters_or,filters_and,[])
        query = get_query_v2(filters_or,[],[])

        #response = list()
        try:
            a = ES_CLIENT.search(body=query,scroll='10s', request_timeout=60,index=index_name,size=1000)['hits']['hits']
            #listout.append([[x['_source']] for x in a])
            for i in a:
                listout = listout + [i['_source']]
        except:
            import traceback 
            print(traceback.print_exc())
            pass

    #     for i in ES_CLIENT.itersearch(body=query,scroll='10s', request_timeout=60,index=index_name,size=1):
    #         #i[0]['_source']['brand_name'] = line['brand_name']
    #         listout = listout + [i[0]['_source']]
        #print (rangex,rangey)
    final_df = pd.DataFrame(listout)
    return final_df

In [7]:
client = MongoClient('10.240.0.144',36018)
client.admin.authenticate('yogesh.kumar', 'DEBdcydbedebhf', mechanism='SCRAM-SHA-1')
#DB_NAME = 'coba_sample_1'
DB_NAME = 'Drugs'
db = client[DB_NAME]
coll_train = db['all_training_data']
dfdata = pd.DataFrame(list(coll_train.find()))
print(dfdata.shape)

# drop trials without drugs
dfdata = dfdata[~dfdata['Experimental_drugs'].isnull()]
print(dfdata.shape)

(35105, 30)
(30913, 30)


In [8]:
# define test set
listofOOS = ['NCT01146561','NCT01127893','NCT00924664','NCT00784693','NCT00960804','NCT00863772','NCT00809783','NCT01087203','NCT00999518','NCT00809354','NCT02528253','NCT02697773','NCT00994890','NCT01089725','NCT00985621','NCT00744471','NCT00545129','NCT02709486','NCT02528188','NCT02609828']
len(listofOOS)

20

In [9]:
dfdata['created_at_timestamp'] = dfdata['created_at_timestamp'].apply(lambda x: int(float(x)))
dfdata['created_at'].value_counts()

2019-02-13    21549
2019-02-18     4817
2019-02-25     1720
2019-03-15     1349
2019-05-13      724
2019-03-04      465
2019-05-08      289
Name: created_at, dtype: int64

In [10]:
file = open("Data/Priyank_data/df_final/df_final_12mar.pkl",'rb')
df_final = pickle.load(file)
file.close()
print(df_final.shape)

file = open("Data/Priyank_data/df_final/df_final_20_Vatsal_NCTS_2_May.pkl",'rb')
df_final1 = pickle.load(file)
file.close()
print(df_final1.shape)

df_final = pd.concat([df_final,df_final1])
print(df_final.shape)

df_final[df_final['clinical_id'].isin(listofOOS)].clinical_id.nunique()

(146183, 268)
(652, 264)
(146835, 268)


20

In [19]:
df_final.shape

(146835, 268)

In [11]:
df_final.phase.value_counts()

phase 2            54662
phase 3            48812
phase 1            16324
phase 4             9543
others              8314
n/a                 2633
phase 1/2           2084
phase 1/phase 2     1821
phase 2/phase 3     1626
phase 2/3            908
phase 0              101
early phase 1          7
Name: phase, dtype: int64

In [12]:
df_final.groupby('phase').val.value_counts()

phase            val
early phase 1    1          4
                 0          3
n/a              0       1696
                 1        937
others           2       3988
                 0       2172
                 1       2154
phase 0          0         59
                 2         22
                 1         20
phase 1          2      11196
                 1       2946
                 0       2182
phase 1/2        0       1383
                 2        367
                 1        334
phase 1/phase 2  0       1511
                 1        302
                 2          8
phase 2          0      25907
                 2      15936
                 1      12819
phase 2/3        0        531
                 1        338
                 2         39
phase 2/phase 3  0       1125
                 1        481
                 2         20
phase 3          1      20411
                 0      16469
                 2      11932
phase 4          2       4198
                 0 

In [18]:
df_final.describe()

,Combination_Therapy,Comparison_Therapy,DrugGroup__approved,DrugGroup__experimental,DrugGroup__illicit,DrugGroup__investigational,DrugGroup__nutraceutical,DrugGroup__vet_approved,DrugGroup__withdrawn,KOL,Medium_aerosol,Medium_capsule,Medium_injection,Medium_kit,Medium_lotion,Medium_other,Medium_patch,Medium_powder,Medium_solution,Medium_suspension,Medium_tablet,Monotherapy,TA_class_TA_cardiology,TA_class_TA_dermatology,TA_class_TA_endocrinology_metabolic,TA_class_TA_gastroenterology,TA_class_TA_genetic,TA_class_TA_hematology,TA_class_TA_immunology,TA_class_TA_infections,TA_class_TA_musculoskeletal,TA_class_TA_nephrology,TA_class_TA_neurology,TA_class_TA_obstetrics_gynaecology,TA_class_TA_oncology,TA_class_TA_ophthalmology,TA_class_TA_others,TA_class_TA_respiratory,TA_class_TA_urology,approved_drug_count_by_sponsor,armgrpclass_NoIntervention,armgrpclass_Placebo,armgrpclass_Sham,armgrpclass_active,armgrpclass_experimental,aromatic_rings_count,authorclass_princiinvest,authorclass_sponsorinvestigator,authorclass_studychair,authorclass_studydir,biological_IC,biological_enzyme,biological_gpcr,biological_kinase,biological_nr,biological_transporter,biological_unspecified,completiondateclass_actual_completion,completiondateclass_anticipated_completion,condition_history_phase1,condition_history_phase1_com,condition_history_phase1_term,condition_history_phase2,condition_history_phase2_com,condition_history_phase2_term,condition_history_phase3,condition_history_phase3_com,condition_history_phase3_term,condition_history_phase4,condition_history_phase4_com,condition_history_phase4_term,condition_sponsor_history_phase1,condition_sponsor_history_phase1_com,condition_sponsor_history_phase1_term,condition_sponsor_history_phase2,condition_sponsor_history_phase2_com,condition_sponsor_history_phase2_term,condition_sponsor_history_phase3,condition_sponsor_history_phase3_com,condition_sponsor_history_phase3_term,condition_sponsor_history_phase4,condition_sponsor_history_phase4_com,condition_sponsor_history_phase4_term,cong_drugcond_no_of_congresses,count_of_drugs_exp,created_at_ES,diff_in_completiondate,drug_condition_history_phase1,drug_condition_history_phase1_com,drug_condition_history_phase1_term,drug_condition_history_phase2,drug_condition_history_phase2_com,drug_condition_history_phase2_term,drug_condition_history_phase3,drug_condition_history_phase3_com,drug_condition_history_phase3_term,drug_condition_history_phase4,drug_condition_history_phase4_com,drug_condition_history_phase4_term,drug_history_phase1,drug_history_phase1_com,drug_history_phase1_term,drug_history_phase2,drug_history_phase2_com,drug_history_phase2_term,drug_history_phase3,drug_history_phase3_com,drug_history_phase3_term,drug_history_phase4,drug_history_phase4_com,drug_history_phase4_term,drug_sponsor_history_phase1,drug_sponsor_history_phase1_com,drug_sponsor_history_phase1_term,drug_sponsor_history_phase2,drug_sponsor_history_phase2_com,drug_sponsor_history_phase2_term,drug_sponsor_history_phase3,drug_sponsor_history_phase3_com,drug_sponsor_history_phase3_term,drug_sponsor_history_phase4,drug_sponsor_history_phase4_com,drug_sponsor_history_phase4_term,duration_in_days,endpt_classifictn_effectiveness,endpt_classifictn_efficacy,endpt_classifictn_pharmacokinetics,endpt_classifictn_safety,enrollment_num,gender_proper_Both,gender_proper_Female,gender_proper_Male,heavy_atoms_count,hydrogen_bond_acceptor_count,hydrogen_bond_donor_count,if_all_drugs_for_some_indication,if_any_drug_prior_approved,indication_approved_for_all_drugs,intervention_Crossover_Assignment,intervention_Factorial_Assignment,intervention_None,intervention_Parallel_Assignment,intervention_Sequential_Assignment,intervention_Single_Group_Assignment,isTop20,is_rare_disease,logp_value,masking_doubleblind,masking_openlabel,masking_singleblind,maximum_age_month,minimum_age_month,molecular_weight,no_of_drugs_approved_for_indication,no_of_grants_condition,no_of_grants_drug,no_of_grants_drug_condition,no_of_manufacturer,

In [61]:
df_final[:2]

,Combination_Therapy,Comparison_Therapy,DrugBank_Entry,DrugGroup__approved,DrugGroup__experimental,DrugGroup__illicit,DrugGroup__investigational,DrugGroup__nutraceutical,DrugGroup__vet_approved,DrugGroup__withdrawn,Drugs_Normalized,KOL,Mapped_Drug,Medium_aerosol,Medium_capsule,Medium_injection,Medium_kit,Medium_lotion,Medium_other,Medium_patch,Medium_powder,Medium_solution,Medium_suspension,Medium_tablet,Monotherapy,TA_class_TA_cardiology,TA_class_TA_dermatology,TA_class_TA_endocrinology_metabolic,TA_class_TA_gastroenterology,TA_class_TA_genetic,TA_class_TA_hematology,TA_class_TA_immunology,TA_class_TA_infections,TA_class_TA_musculoskeletal,TA_class_TA_nephrology,TA_class_TA_neurology,TA_class_TA_obstetrics_gynaecology,TA_class_TA_oncology,TA_class_TA_ophthalmology,TA_class_TA_others,TA_class_TA_respiratory,TA_class_TA_urology,_id,application_number,approved_drug_count_by_sponsor,armgrpclass_NoIntervention,armgrpclass_Placebo,armgrpclass_Sham,armgrpclass_active,armgrpclass_experimental,aromatic_rings_count,authorclass_princiinvest,authorclass_sponsorinvestigator,authorclass_studychair,authorclass_studydir,biological_IC,biological_enzyme,biological_gpcr,biological_kinase,biological_nr,biological_transporter,biological_unspecified,clinical_id,completiondateclass_actual_completion,completiondateclass_anticipated_completion,condition,condition_history_phase1,condition_history_phase1_com,condition_history_phase1_term,condition_history_phase2,condition_history_phase2_com,condition_history_phase2_term,condition_history_phase3,condition_history_phase3_com,condition_history_phase3_term,condition_history_phase4,condition_history_phase4_com,condition_history_phase4_term,condition_sponsor_history_phase1,condition_sponsor_history_phase1_com,condition_sponsor_history_phase1_term,condition_sponsor_history_phase2,condition_sponsor_history_phase2_com,condition_sponsor_history_phase2_term,condition_sponsor_history_phase3,condition_sponsor_history_phase3_com,condition_sponsor_history_phase3_term,condition_sponsor_history_phase4,condition_sponsor_history_phase4_com,condition_sponsor_history_phase4_term,cong_drugcond_no_of_congresses,count_of_drugs_exp,created_at,created_at_ES,created_at_timestamp,diff_in_completiondate,drug_condition_history_phase1,drug_condition_history_phase1_com,drug_condition_history_phase1_term,drug_condition_history_phase2,drug_condition_history_phase2_com,drug_condition_history_phase2_term,drug_condition_history_phase3,drug_condition_history_phase3_com,drug_condition_history_phase3_term,drug_condition_history_phase4,drug_condition_history_phase4_com,drug_condition_history_phase4_term,drug_history_phase1,drug_history_phase1_com,drug_history_phase1_term,drug_history_phase2,drug_history_phase2_com,drug_history_phase2_term,drug_history_phase3,drug_history_phase3_com,drug_history_phase3_term,drug_history_phase4,drug_history_phase4_com,drug_history_phase4_term,drug_sponsor_history_phase1,drug_sponsor_history_phase1_com,drug_sponsor_history_phase1_term,drug_sponsor_history_phase2,drug_sponsor_history_phase2_com,drug_sponsor_history_phase2_term,drug_sponsor_history_phase3,drug_sponsor_history_phase3_com,drug_sponsor_history_phase3_term,drug_sponsor_history_phase4,drug_sponsor_history_phase4_com,drug_sponsor_history_phase4_term,duration_in_days,endpt_classifictn_effectiveness,endpt_classifictn_efficacy,endpt_classifictn_pharmacokinetics,endpt_classifictn_safety,enrollment_num,gender_proper_Both,gender_proper_Female,gender_proper_Male,heavy_atoms_count,hydrogen_bond_acceptor_count,hydrogen_bond_donor_count,id_info,if_all_drugs_for_some_indication,if_any_drug_prior_approved,indication_approved_for_all_drugs,intervention_Crossover_Assignment,intervention_Factorial_Assignment,intervention_None,intervention_Parallel_Assignment,intervention_Sequential_Assignment,intervention_Single_Group_Assignment,isTop20,is_rare_disease,lastchanged_date,logp_value,masking_doubleblind,masking_openlabel,masking_singleblind,maximum_age_month,minimum_age_m

In [59]:
# # get distribution by year for WorldQuant

# df_final = df_final[df_final['val'] != 2]

# df_final

# df = df_final[['clinical_id','val','version', 'lastchanged_date']]

# df['lastchanged_date'].iloc[17]

# df['year'] = df['lastchanged_date'].apply(lambda x: x[:4] if isinstance(x, str) else 0)

In [86]:
df['year'].value_counts()

0       25687
2016    22345
2015     5068
2013     4842
2012     4810
2014     4785
2017     4566
2009     4300
2011     4300
2010     4215
2018     4020
2008     3706
2007     3017
2006     2060
2005     1137
2019      243
           28
Name: year, dtype: int64

In [84]:
# df[df['year'] == '2014'].clinical_id.nunique()

# df_final[df_final['clinical_id'].isin(listofOOS)].shape

# del df_final['val']
# print(df_final.shape)
# df_final = pd.merge(df_final,dfdata[['clinical_id','val']],on='clinical_id',how='left')
# print(df_final.shape)

# print(df_final.val.isnull().sum())

4257

# Concat all df and drop duplicates

In [9]:
df_final.drop_duplicates(subset=['clinical_id','version'],inplace=True)
print(df_final.shape)

df_final.sort_values(['clinical_id'], inplace = True)
df_final.reset_index(inplace = True,drop=True)

(146690, 268)


In [11]:
df_final[df_final['clinical_id'].isin(listofOOS)].shape

(652, 268)

In [12]:
#Read columns to be taken from 'on_the_fly' collection
df_columns = pd.read_csv('/data/users-workspace/anshul.jain/code/clinical_trial/phase_prediction/Data/Training_data_column_names.csv')
print(df_columns.shape)
df_columns[:2]

model_features = df_columns['model_features'].tolist()
len(model_features)

# model_features = [x for x in model_features if x not in remove_feats]
# len(model_features)

(246, 3)


246

In [13]:
print(df_final.shape)
df_final.dropna(subset = ['val'], inplace = True)
print(df_final.shape)

(146690, 268)
(132354, 268)


In [14]:
df_final_gbm = df_final.copy()
print(df_final_gbm.shape)
# df_final_test_gbm = df_final_test.copy()
# print(df_final_test_gbm.shape)

(132354, 268)


In [23]:
# df_final = df_final_gbm.copy()

In [349]:
df_final[df_final['clinical_id'].isin(listofOOS)].shape

(652, 268)

# Features for NN

In [24]:
# split test and train set into sets of categorical and numeric features
#test_features = df_final[df_final['clinical_id'].isin(listofOOS)]
test_features = df_final[df_final['clinical_id'].isin(listofOOS)]
train_features = df_final[~(df_final['clinical_id'].isin(listofOOS))]
train_features = train_features[~(train_features['val'] == 2)]

print(test_features.shape)
print(train_features.shape)
test_features.clinical_id.nunique()
# test_features[test_features['val']!=2]['clinical_id'].nunique()

train_features[train_features['val'].notnull()].shape
train_features[train_features['val'].notnull()]['clinical_id'].nunique()

test_clinical_id = test_features['clinical_id'].tolist()
test_version = test_features['version'].tolist()
test_lastchanged_date = test_features['lastchanged_date'].tolist()
test_overall_status_lower = test_features['overall_status_lower'].tolist()
test_year = test_features['year'].tolist()
len(test_year)

(652, 268)
(88466, 268)


652

In [498]:
df_final[df_final['clinical_id'].isin(listofOOS)].shape

(652, 268)

In [21]:
# df_final2 = df_final.copy()

In [25]:
model_features = [m for m in model_features if m not in ['primary_measure']]
df_final = df_final[model_features]
df_final.shape

(132354, 245)

In [26]:
# merge cols by the overarching characteristic they're describing
df_final['impact_factor'] = df_final['pub_cond_impact_factor']+df_final['pub_cond_sjr_impact_factor']+df_final['pub_drugcond_impact_factor']+df_final['pub_drugcond_sjr_impact_factor']+df_final['pub_drug_impact_factor']+df_final['pub_drug_sjr_impact_factor']
df_final['no_of_publications'] = df_final['pub_cond_no_of_publications']+df_final['pub_drugcond_no_of_publications']+df_final['pub_drug_no_of_publications']
df_final['no_of_grants'] = df_final['no_of_grants_drug_condition']+df_final['no_of_grants_drug']+df_final['no_of_grants_condition']
df_final['no_of_patents'] = df_final['no_of_patents_drug_condition']+df_final['no_of_patents_drug']+df_final['no_of_patents_condition']
df_final['drug_condition_history_phase1'] = df_final['drug_history_phase1']+df_final['drug_history_phase1_com']+df_final['drug_history_phase1_term']+df_final['condition_history_phase1']+df_final['condition_history_phase1_com']+df_final['condition_history_phase1_term']
df_final['drug_condition_history_phase2'] = df_final['drug_history_phase2']+df_final['drug_history_phase2_com']+df_final['drug_history_phase2_term']+df_final['condition_history_phase2']+df_final['condition_history_phase2_com']+df_final['condition_history_phase2_term']
df_final['drug_condition_history_phase3'] = df_final['drug_history_phase3']+df_final['drug_history_phase3_com']+df_final['drug_history_phase3_term']+df_final['condition_history_phase3']+df_final['condition_history_phase3_com']+df_final['condition_history_phase3_term']
df_final['drug_condition_history_phase4'] = df_final['drug_history_phase4']+df_final['drug_history_phase4_com']+df_final['drug_history_phase4_term']+df_final['condition_history_phase4']+df_final['condition_history_phase4_com']+df_final['condition_history_phase4_term']
df_final['sponsor_history_phase1'] = df_final['sponsor_history_phase1']+df_final['sponsor_history_phase1_com']+df_final['sponsor_history_phase1_term']+df_final['drug_sponsor_history_phase1']+df_final['drug_sponsor_history_phase1_com']+df_final['drug_sponsor_history_phase1_term']+df_final['condition_sponsor_history_phase1']+df_final['condition_sponsor_history_phase1_com']+df_final['condition_sponsor_history_phase1_term']
df_final['sponsor_history_phase2'] = df_final['sponsor_history_phase2']+df_final['sponsor_history_phase2_com']+df_final['sponsor_history_phase2_term']+df_final['drug_sponsor_history_phase2']+df_final['drug_sponsor_history_phase2_com']+df_final['drug_sponsor_history_phase2_term']+df_final['condition_sponsor_history_phase2']+df_final['condition_sponsor_history_phase2_com']+df_final['condition_sponsor_history_phase2_term']
df_final['sponsor_history_phase3'] = df_final['sponsor_history_phase3']+df_final['sponsor_history_phase3_com']+df_final['sponsor_history_phase3_term']+df_final['drug_sponsor_history_phase3']+df_final['drug_sponsor_history_phase3_com']+df_final['drug_sponsor_history_phase3_term']+df_final['condition_sponsor_history_phase3']+df_final['condition_sponsor_history_phase3_com']+df_final['condition_sponsor_history_phase3_term']
df_final['sponsor_history_phase4'] = df_final['sponsor_history_phase4']+df_final['sponsor_history_phase4_com']+df_final['sponsor_history_phase4_term']+df_final['drug_sponsor_history_phase4']+df_final['drug_sponsor_history_phase4_com']+df_final['drug_sponsor_history_phase4_term']+df_final['condition_sponsor_history_phase4']+df_final['condition_sponsor_history_phase4_com']+df_final['condition_sponsor_history_phase4_term']

In [27]:
df_final.drop(['pub_cond_impact_factor', 'pub_cond_no_of_publications', 'pub_cond_sjr_impact_factor', 'pub_drugcond_impact_factor', 'pub_drugcond_no_of_publications', 'pub_drugcond_sjr_impact_factor', 'pub_drug_impact_factor', 'pub_drug_no_of_publications', 'pub_drug_sjr_impact_factor', 'no_of_grants_drug_condition', 'no_of_grants_drug', 'no_of_grants_condition', 'no_of_patents_drug_condition', 'no_of_patents_drug', 'no_of_patents_condition', 'drug_history_phase1', 'drug_history_phase1_com', 'drug_history_phase1_term', 'drug_history_phase2', 'drug_history_phase2_com', 'drug_history_phase2_term', 'drug_history_phase3', 'drug_history_phase3_com', 'drug_history_phase3_term', 'drug_history_phase4', 'drug_history_phase4_com', 'drug_history_phase4_term', 'condition_history_phase1', 'condition_history_phase1_com', 'condition_history_phase1_term', 'condition_history_phase2', 'condition_history_phase2_com', 'condition_history_phase2_term', 'condition_history_phase3', 'condition_history_phase3_com', 'condition_history_phase3_term', 'condition_history_phase4', 'condition_history_phase4_com', 'condition_history_phase4_term', 'drug_condition_history_phase1_com', 'drug_condition_history_phase1_term', 'drug_condition_history_phase2_com', 'drug_condition_history_phase2_term', 'drug_condition_history_phase3_com', 'drug_condition_history_phase3_term', 'drug_condition_history_phase4_com', 'drug_condition_history_phase4_term', 'sponsor_history_phase1_com', 'sponsor_history_phase1_term', 'sponsor_history_phase2_com', 'sponsor_history_phase2_term', 'sponsor_history_phase3_com', 'sponsor_history_phase3_term', 'sponsor_history_phase4_com', 'sponsor_history_phase4_term', 'drug_sponsor_history_phase1', 'drug_sponsor_history_phase1_com', 'drug_sponsor_history_phase1_term', 'drug_sponsor_history_phase2', 'drug_sponsor_history_phase2_com', 'drug_sponsor_history_phase2_term', 'drug_sponsor_history_phase3', 'drug_sponsor_history_phase3_com', 'drug_sponsor_history_phase3_term', 'drug_sponsor_history_phase4', 'drug_sponsor_history_phase4_com', 'drug_sponsor_history_phase4_term', 'condition_sponsor_history_phase1', 'condition_sponsor_history_phase1_com', 'condition_sponsor_history_phase1_term', 'condition_sponsor_history_phase2', 'condition_sponsor_history_phase2_com', 'condition_sponsor_history_phase2_term', 'condition_sponsor_history_phase3', 'condition_sponsor_history_phase3_com', 'condition_sponsor_history_phase3_term', 'condition_sponsor_history_phase4', 'condition_sponsor_history_phase4_com', 'condition_sponsor_history_phase4_term'], inplace = True, axis = 1)
df_final.reset_index(inplace = True, drop = True)

In [28]:
df_final.drop(['no_of_drugs_approved_for_indication',
    'drug_condition_history_phase1',
    'drug_condition_history_phase2',
    'drug_condition_history_phase3',
    'drug_condition_history_phase4',
    'impact_factor',
    'sponsor_history_phase1',
    'sponsor_history_phase2',
    'sponsor_history_phase3',
    'no_of_grants',
    'no_of_grants'
              ], inplace = True, axis = 1)

In [20]:
df_final[df_final['clinical_id'].isin(listofOOS)]['clinical_id'].nunique()

20

# Run the below cell for selecting which trials with what 'overall_status' will go into the model

In [30]:
l = train_features[train_features['overall_status_lower'] == 'active, not recruiting'].clinical_id.tolist()
m = train_features[train_features['overall_status_lower'] == 'recruiting'].clinical_id.tolist()
n = train_features[train_features['overall_status_lower'] == 'not yet recruiting'].clinical_id.tolist()

o = list(set(l+m+n))
# o = o[:int(len(o)/2)]
len(o)

19924

In [31]:
k = df_final[(df_final['val'] == 0) & (df_final['clinical_id'].isin(o))].sample(3000).clinical_id.unique()
print(len(k))
df_final = df_final[~(df_final['clinical_id'].isin(k))]
df_final.val.value_counts()

2592


2.0    43462
1.0    40054
0.0    35699
Name: val, dtype: int64

In [32]:
# split test and train set into sets of categorical and numeric features
#test_features = df_final[df_final['clinical_id'].isin(listofOOS)]
test_features = df_final[df_final['clinical_id'].isin(listofOOS)]
train_features = df_final[~(df_final['clinical_id'].isin(listofOOS))]
train_features = train_features[~(train_features['val'] == 2)]

print(test_features.shape)
print(train_features.shape)

# just retain the intermediate versions in the training set
# terminated, completed, withdrawn, suspended
print(train_features.shape)
train_features = train_features[~((train_features['clinical_id'].isin(o)) & (train_features['overall_status_lower'] == 'terminated'))]
print(train_features.shape)
train_features = train_features[~(train_features['clinical_id'].isin(o) & (train_features['overall_status_lower'] == 'completed'))]
print(train_features.shape)
train_features = train_features[~(train_features['clinical_id'].isin(o) & (train_features['overall_status_lower'] == 'withdrawn'))]
print(train_features.shape)

(652, 160)
(75327, 160)
(75327, 160)
(68749, 160)
(49928, 160)
(45817, 160)


In [33]:
train_features.drop_duplicates(subset = ['clinical_id'], inplace = True, keep = 'first')
print(train_features.shape)

train_features.groupby(['overall_status_lower']).val.value_counts()

(19681, 160)


overall_status_lower     val
                         1.0       7
active, not recruiting   1.0    3517
                         0.0    2049
approved for marketing   1.0       9
available                0.0       1
                         1.0       1
completed                1.0    1227
                         0.0     288
enrolling by invitation  0.0      82
                         1.0      56
no longer available      1.0       2
                         0.0       1
not yet recruiting       0.0    2311
                         1.0     578
recruiting               0.0    4522
                         1.0    3877
suspended                0.0     411
                         1.0      37
terminated               0.0     525
                         1.0      17
withdrawn                0.0     163
Name: val, dtype: int64

In [112]:
# assert no test NCTs present in the training data
test_features[test_features['clinical_id'].isin(train_features['clinical_id'].tolist())].shape

(0, 152)

In [36]:
numcols = ['clinical_id',

 'number_of_arms',
 'num_of_countries',
 'num_of_sites',
 'number_of_collaborators',
 'minimum_age_month',
 'maximum_age_month',
 'duration_in_days',
 'trials_in_2_years',
 'enrollment_num',
 'number_of_authors',
 'number_of_princi_invest',
 'authorclass_princiinvest',
 'authorclass_studydir',
 'authorclass_studychair',
 'trial_activity_duration',
 'KOL',
 'count_of_drugs_exp',
#  'drug_condition_history_phase1',
#  'drug_condition_history_phase2',
#  'drug_condition_history_phase3',
#  'drug_condition_history_phase4',
 'cong_drugcond_no_of_congresses',
 'approved_drug_count_by_sponsor',
 'diff_in_completiondate',
 'indication_approved_for_all_drugs',
# 'no_of_drugs_approved_for_indication',
 'variations_in_completiondate_using_versions',
 'polar_surface_area',
 'hydrogen_bond_acceptor_count',
 'hydrogen_bond_donor_count',
 'violations_in_octet_rule_count',
 'heavy_atoms_count',
 'logp_value',
 'rotatable_bond_count',
 'aromatic_rings_count',
 'molecular_weight',
 'no_of_metabolites_incr',
 'no_of_metabolites_decr',
 'no_of_targets',
 'no_of_pathways',
 'no_of_manufacturer',
#  'impact_factor',
 'no_of_publications',
#  'no_of_grants',
#  'no_of_patents'
          ]
           
numdf = df_final[numcols]
catcols = list(set(df_final.columns.tolist()) ^ set(numcols)) + ['clinical_id']

rem = ['os_activenotrecruiting', 'os_completed', 'os_recruiting', 'os_withdrawn', 'os_notyetrecruiting', 'os_random', 'os_suspended', 'os_terminated','os_enrollingbyinvitation']
catcols = [i for i in catcols if i not in rem]
catdf = df_final[catcols]

rem = rem + ['clinical_id']
ovldf = df_final[rem]

testnum = test_features[numcols]
testcat = test_features[catcols]
testovl = test_features[rem]

trainnum = train_features[numcols]
traincat = train_features[catcols]
trainovl = train_features[rem]

print(testnum.shape)
print(testcat.shape)
print(testovl.shape)
print(trainnum.shape)
print(traincat.shape)
print(trainovl.shape)

(652, 38)
(652, 114)
(652, 10)
(19681, 38)
(19681, 114)
(19681, 10)


In [37]:
# drop unnecessary and sparse cols 
train_labels = np.array(train_features['val'])
trainnum = np.array(trainnum.drop(['hydrogen_bond_acceptor_count','aromatic_rings_count',"clinical_id"],axis=1))
trainnum = trainnum.astype(float)
# traincat = np.array(traincat.drop(["index","val","clinical_id",'overall_status_lower', 'purpose_None'],axis=1))
traincat = np.array(traincat.drop(["val","clinical_id",'overall_status_lower', 'purpose_None'],axis=1))
traincat = traincat.astype(float)
trainovl = np.array(trainovl.drop(["clinical_id"],axis=1))
trainovl = trainovl.astype(float)

test_labels=np.array(test_features['val'])
testnum = np.array(testnum.drop(['hydrogen_bond_acceptor_count','aromatic_rings_count','clinical_id'],axis=1))
testnum = testnum.astype(float)
# testcat = np.array(testcat.drop(["index","val","clinical_id",'overall_status_lower', 'purpose_None'],axis=1))
testcat = np.array(testcat.drop(["val","clinical_id",'overall_status_lower', 'purpose_None'],axis=1))
testcat = testcat.astype(float)
testovl = np.array(testovl.drop(["clinical_id"],axis=1))
testovl = testovl.astype(float)

train_one_hot = to_categorical(train_labels)
print(train_labels.shape, trainnum.shape, traincat.shape, trainovl.shape, test_labels.shape, testnum.shape, testcat.shape, testovl.shape)

test_features1 = test_features[test_features["val"]==2]
test_features2 = test_features[~(test_features["val"]==2)]
print(test_features1.shape)
print(test_features2.shape)

testnum1 = test_features1[numcols]
testcat1 = test_features1[catcols]
testovl1 = test_features1[rem]

testnum2 = test_features2[numcols]
testcat2 = test_features2[catcols]
testovl2 = test_features2[rem]

test_labels1=np.array(test_features1['val'])
testnum1 = np.array(testnum1.drop(['hydrogen_bond_acceptor_count','aromatic_rings_count','clinical_id'],axis=1))
testnum1 = testnum1.astype(float)
# testcat1 = np.array(testcat1.drop(["index","val","clinical_id",'overall_status_lower', 'purpose_None'],axis=1))
testcat1 = np.array(testcat1.drop(["val","clinical_id",'overall_status_lower', 'purpose_None'],axis=1))
testcat1 = testcat1.astype(float)
testovl1 = np.array(testovl1.drop(["clinical_id"],axis=1))
testovl1 = testovl1.astype(float)

test_labels2=np.array(test_features2['val'])
testnum2 = np.array(testnum2.drop(['hydrogen_bond_acceptor_count','aromatic_rings_count','clinical_id'],axis=1))
testnum2 = testnum2.astype(float)
# testcat2 = np.array(testcat2.drop(["index","val","clinical_id",'overall_status_lower', 'purpose_None'],axis=1))
testcat2 = np.array(testcat2.drop(["val","clinical_id",'overall_status_lower', 'purpose_None'],axis=1))
testcat2 = testcat2.astype(float)
testovl2 = np.array(testovl2.drop(["clinical_id"],axis=1))
testovl2 = testovl2.astype(float)

(19681,) (19681, 35) (19681, 110) (19681, 9) (652,) (652, 35) (652, 110) (652, 9)
(226, 160)
(426, 160)


In [38]:
from keras.callbacks import Callback
import keras.backend as K
import numpy as np

class SGDRScheduler(Callback):
    '''Cosine annealing learning rate scheduler with periodic restarts.

    # Usage
        ```python
            schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=1e-2,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.5)
            model.fit(X_train, Y_train, epochs=100, callbacks=[schedule])
        ```

    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        lr_decay: Reduce the max_lr after the completion of each cycle.
                  Ex. To reduce the max_lr by 20% after each cycle, set this value to 0.8.
        cycle_length: Initial number of epochs in a cycle.
        mult_factor: Scale epochs_to_restart after each full cycle completion.

    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: http://arxiv.org/abs/1608.03983
    '''
    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=10,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)

In [39]:
# feed numeric features
train_input1 = Input(shape = (trainnum.shape[1], ), name = "input_train1")
train_norm1 = BatchNormalization(name = 'norm_train1')(train_input1)
dense_layer1_1 = Dense(8 , kernel_regularizer=regularizers.l2(0.01),activation = 'relu')(train_norm1)
dense_layer1 = Lambda(lambda x: x * 2)(dense_layer1_1)

# feed categorical features
train_input2 = Input(shape = (traincat.shape[1], ), name = "input_train2")
train_norm2 = BatchNormalization(name = 'norm_train2')(train_input2)
dense_layer2 = Dense(8 , kernel_regularizer=regularizers.l2(0.01), activation = 'relu')(train_norm2)

# feed overall status
train_input3 = Input(shape = (trainovl.shape[1], ), name = "input_train3")
train_norm3 = BatchNormalization(name = 'norm_train3')(train_input3)
dense_layer3_3 = Dense(8 , activation = 'relu')(train_norm3)
dense_layer_ovl = Lambda(lambda x: x * 1)(dense_layer3_3)

# concat output of concat layer with one from the above dense layer_
input_concat = concatenate([dense_layer1, dense_layer2, dense_layer_ovl] , name = 'concantenate_layer')
# input_concat = concatenate([dense_layer1, dense_layer2] , name = 'concantenate_layer')
dropout_layer1 = Dropout((0.3) , name = 'dropout_layer1')(input_concat)
dense_layer3 = Dense(16, kernel_regularizer=regularizers.l2(0.1),activation = 'relu')(dropout_layer1) #16

train_norm4 = BatchNormalization(name = 'norm_train4')(dense_layer3)
dropout_layer2 = Dropout((0.5) , name = 'dropout_layer2')(train_norm4)
dense_layer4 = Dense(8, kernel_regularizer=regularizers.l2(0.01), activation = 'relu')(dropout_layer2) #8
dense_layer5 = Dense(4, activation = 'relu')(dense_layer4)
output = Dense(1, activation = 'sigmoid')(dense_layer5)

print("Model architecture defined")
lol = Model(inputs = [train_input1, train_input2, train_input3] , outputs = output)
lol.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['binary_accuracy'])
#print(lol.summary())

file_path="/data/users-workspace/anshul.jain/code/clinical_trial/phase_prediction/Data/Priyank_data/Feed_ForwardNN_Priyank_16mar_28feb_pickle.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_binary_accuracy', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=100)
callbacks_list = [checkpoint, early]
epoch_size = 500
batch_size = 64
schedule = SGDRScheduler(min_lr=1e-5, max_lr=1e-2, steps_per_epoch=np.ceil(epoch_size/batch_size), lr_decay=0.7, cycle_length=5, mult_factor=1.5)

Model architecture defined


In [520]:
df_final2.to_csv("feats.csv")

In [40]:
history = lol.fit([trainnum, traincat, trainovl] , train_labels, validation_split=0.2, shuffle = True , epochs = epoch_size, batch_size = batch_size , callbacks=[schedule, checkpoint, early])# sample_weight = sw_train)

Train on 15744 samples, validate on 3937 samples
Epoch 1/500
15744/15744 [==============================] - 4s 255us/step - loss: 0.8981 - binary_accuracy: 0.6414 - val_loss: 0.6826 - val_binary_accuracy: 0.6525

Epoch 00001: val_binary_accuracy improved from -inf to 0.65253, saving model to /data/users-workspace/anshul.jain/code/clinical_trial/phase_prediction/Data/Priyank_data/Feed_ForwardNN_Priyank_16mar_28feb_pickle.hdf5
Epoch 2/500
15744/15744 [==============================] - 2s 101us/step - loss: 0.6314 - binary_accuracy: 0.7027 - val_loss: 0.6314 - val_binary_accuracy: 0.6749

Epoch 00002: val_binary_accuracy improved from 0.65253 to 0.67488, saving model to /data/users-workspace/anshul.jain/code/clinical_trial/phase_prediction/Data/Priyank_data/Feed_ForwardNN_Priyank_16mar_28feb_pickle.hdf5
Epoch 3/500
15744/15744 [==============================] - 1s 94us/step - loss: 0.6180 - binary_accuracy: 0.7075 - val_loss: 0.6055 - val_binary_accuracy: 0.7132

Epoch 00003: val_binary_a

15744/15744 [==============================] - 2s 103us/step - loss: 0.5692 - binary_accuracy: 0.7254 - val_loss: 0.5755 - val_binary_accuracy: 0.6939

Epoch 00033: val_binary_accuracy did not improve from 0.72924
Epoch 34/500
15744/15744 [==============================] - 2s 97us/step - loss: 0.5677 - binary_accuracy: 0.7266 - val_loss: 0.5701 - val_binary_accuracy: 0.7089

Epoch 00034: val_binary_accuracy did not improve from 0.72924
Epoch 35/500
15744/15744 [==============================] - 2s 97us/step - loss: 0.5702 - binary_accuracy: 0.7245 - val_loss: 0.5594 - val_binary_accuracy: 0.7173

Epoch 00035: val_binary_accuracy did not improve from 0.72924
Epoch 36/500
15744/15744 [==============================] - 1s 93us/step - loss: 0.5667 - binary_accuracy: 0.7236 - val_loss: 0.5576 - val_binary_accuracy: 0.7198

Epoch 00036: val_binary_accuracy did not improve from 0.72924
Epoch 37/500
15744/15744 [==============================] - 1s 79us/step - loss: 0.5665 - binary_accuracy: 0

15744/15744 [==============================] - 1s 93us/step - loss: 0.5537 - binary_accuracy: 0.7309 - val_loss: 0.5569 - val_binary_accuracy: 0.7160

Epoch 00105: val_binary_accuracy did not improve from 0.72924
Epoch 106/500
15744/15744 [==============================] - 1s 92us/step - loss: 0.5528 - binary_accuracy: 0.7329 - val_loss: 0.5570 - val_binary_accuracy: 0.7130

Epoch 00106: val_binary_accuracy did not improve from 0.72924
Epoch 107/500
15744/15744 [==============================] - 1s 94us/step - loss: 0.5519 - binary_accuracy: 0.7319 - val_loss: 0.5619 - val_binary_accuracy: 0.7089

Epoch 00107: val_binary_accuracy did not improve from 0.72924
Epoch 108/500
15744/15744 [==============================] - 1s 85us/step - loss: 0.5582 - binary_accuracy: 0.7305 - val_loss: 0.5678 - val_binary_accuracy: 0.7054

Epoch 00108: val_binary_accuracy did not improve from 0.72924
Epoch 109/500
15744/15744 [==============================] - 2s 101us/step - loss: 0.5500 - binary_accurac

In [396]:
# model_nn = load_model('/data/users-workspace/anshul.jain/code/clinical_trial/phase_prediction/Data/Priyank_data/Feed_ForwardNN_Priyank_28mar_acatis.hdf5')
# model_nn = load_model('/data/users-workspace/anshul.jain/code/clinical_trial/phase_prediction/Data/Priyank_data/Feed_ForwardNN_Priyank_14mar_acatis_recheck_2_5k.hdf5')
model_nn = load_model('/data/users-workspace/anshul.jain/code/clinical_trial/phase_prediction/Data/Priyank_data/Feed_ForwardNN_Priyank_16mar_28feb_pickle.hdf5')

In [41]:
train_labels = train_labels.astype(int)
print(np.bincount(train_labels))

test_labels2 = test_labels2.astype(int)
print(np.bincount(test_labels2))

[10353  9328]
[354  72]


In [42]:
probs_nn = lol.predict([testnum2, testcat2, testovl2]) 
predicted_nn = [1 if i >= 0.5 else 0 for i in probs_nn]

print ("accuracy is ",metrics.accuracy_score(test_labels2, predicted_nn))
print ("AUC is ",metrics.roc_auc_score(test_labels2, predicted_nn))
print ("confusion matrix: ")
print (metrics.confusion_matrix(test_labels2, predicted_nn))
print ("precision and recall numbers: ")
print (metrics.classification_report(test_labels2, predicted_nn))

accuracy is  0.715962441314554
AUC is  0.49164312617702444
confusion matrix: 
[[294  60]
 [ 61  11]]
precision and recall numbers: 
              precision    recall  f1-score   support

           0       0.83      0.83      0.83       354
           1       0.15      0.15      0.15        72

   micro avg       0.72      0.72      0.72       426
   macro avg       0.49      0.49      0.49       426
weighted avg       0.71      0.72      0.72       426



In [44]:
probs_nn = lol.predict([testnum, testcat, testovl]) 
predicted_nn = [1 if i >= 0.5 else 0 for i in probs_nn]
len(probs_nn)

probs_nn1 = [np.array(x[0]).tolist() for x in probs_nn]
# probs_nn1

# # probs_nn[:50]
# test_clinical_id 
# test_version 
# test_lastchanged_date 
# test_year 
# test_overall_status_lower
# len(test_year)

df_preds_nn = pd.DataFrame({'clinical_id':test_clinical_id,'version':test_version,'overall_status_lower':test_overall_status_lower,'lastchanged_date':test_lastchanged_date,'year':test_year,'probability_1':probs_nn1,'val':test_labels,'prediction':np.array(list(predicted_nn))})
print(df_preds_nn.shape)
df_preds_nn.head()

(652, 8)


,clinical_id,lastchanged_date,overall_status_lower,prediction,probability_1,val,version,year
0,NCT00545129,2009-07-22,recruiting,1,0.668432,0.0,37.0,2009
1,NCT00545129,2010-10-12,recruiting,1,0.659960,0.0,52.0,2010
2,NCT00545129,2010-08-19,recruiting,1,0.659983,0.0,51.0,2010
3,NCT00545129,2009-12-01,recruiting,1,0.668749,0.0,42.0,2009
4,NCT00545129,2009-07-02,recruiting,1,0.668477,0.0,36.0,2009


In [47]:
df_preds_nn[df_preds_nn['clinical_id'] == 'NCT02697773'].sort_values(by = ['version'], ascending = False)

,clinical_id,lastchanged_date,overall_status_lower,prediction,probability_1,val,version,year
578,NCT02697773,2019-04-20,completed,1,0.680545,1.0,38.0,2019
592,NCT02697773,2018-11-01,completed,1,0.653378,1.0,37.0,2018
589,NCT02697773,2018-10-15,completed,1,0.679697,1.0,36.0,2018
579,NCT02697773,2018-10-03,completed,1,0.679395,1.0,35.0,2018
580,NCT02697773,2018-09-05,completed,1,0.680585,1.0,34.0,2018
581,NCT02697773,2018-08-10,completed,1,0.680585,1.0,33.0,2018
593,NCT02697773,2018-05-16,"active, not recruiting",0,0.428328,1.0,32.0,2018
591,NCT02697773,2018-05-07,"active, not recruiting",0,0.457528,1.0,31.0,2018
582,NCT02697773,2018-04-09,"active, not recruiting",0,0.457528,1.0,30.0,2018
583,NCT02697773,2018-02-22,"active, not recruiting",0,0.457528,1.0,29.0,2018


In [421]:
z = df_preds_nn[df_preds_nn['val'] == df_preds_nn['prediction']].clinical_id.unique()

array(['NCT00545129', 'NCT00744471', 'NCT00784693', 'NCT00809354',
       'NCT00809783', 'NCT00863772', 'NCT00924664', 'NCT00960804',
       'NCT00985621', 'NCT00994890', 'NCT00999518', 'NCT01087203',
       'NCT01089725', 'NCT01127893', 'NCT01146561', 'NCT02697773'],
      dtype=object)

In [153]:
df_preds_nn['clinical_id'].nunique()

8018

In [138]:
# df_preds_nn.to_csv('Data/Priyank_data/Acatis_2_5k_NN_predictions_03april.csv',index=False)
df_preds_nn.to_csv('Data/Priyank_data/tanzenumab_nn.csv',index=False)

# Features for GBM

In [432]:
# split test and train set into sets of categorical and numeric features
# test_features_gbm = df_final_gbm[df_final_gbm['clinical_id'].isin(listofOOS)]
test_features_gbm = df_final_gbm[df_final_gbm['clinical_id'].isin(listofOOS)]
train_features_gbm = df_final_gbm[~(df_final_gbm['clinical_id'].isin(listofOOS))]
train_features_gbm = train_features_gbm[~(train_features_gbm['val']==2)]

print(test_features_gbm.shape)
print(train_features_gbm.shape)


(652, 268)
(88466, 268)


In [406]:
# test_features_gbm[test_features_gbm['clinical_id']=="NCT01370642"].describe()

In [ ]:
# test_features_no_version_gbm = df_final_gbm[df_final_gbm['clinical_id'].isin(listofOOS)]
# print(test_features_no_version_gbm.shape)
# test_features_no_version_gbm = test_features_no_version_gbm[~test_features_no_version_gbm['clinical_id'].isin(test_features_gbm['clinical_id'].tolist())]
# print(test_features_no_version_gbm.shape)

In [433]:
# just retain the intermediate versions in the training set
# terminated, completed, withdrawn, suspended
print(train_features_gbm.shape)
train_features_gbm = train_features_gbm[~((train_features_gbm['clinical_id'].isin(o)) & (train_features_gbm['overall_status_lower'] == 'terminated'))]
print(train_features_gbm.shape)
train_features_gbm = train_features_gbm[~(train_features_gbm['clinical_id'].isin(o) & (train_features_gbm['overall_status_lower'] == 'completed'))]
print(train_features_gbm.shape)
train_features_gbm = train_features_gbm[~(train_features_gbm['clinical_id'].isin(o) & (train_features_gbm['overall_status_lower'] == 'withdrawn'))]
print(train_features_gbm.shape)

(88466, 268)
(81664, 268)
(62604, 268)
(58384, 268)


In [434]:
train_features_gbm.drop_duplicates(subset = ['clinical_id'], inplace = True, keep = 'first')
print(train_features_gbm.shape)

(22273, 268)


In [435]:
# assert no test NCTs present in the training data
test_features_gbm[test_features_gbm['clinical_id'].isin(train_features_gbm['clinical_id'].tolist())].shape

(0, 268)

In [436]:
# test_features_gbm['index'] = 0
# model_features = [m for m in model_features if m not in ['type_small_molecule','primary_measure']]
model_features = [m for m in model_features if m not in ['primary_measure']]
train_features_gbm = train_features_gbm[model_features]
train_features_gbm.head(2)

,clinical_id,val,overall_status_lower,number_of_arms,num_of_countries,num_of_sites,number_of_collaborators,minimum_age_month,maximum_age_month,type_of_lead_sponsor_Hospital,type_of_lead_sponsor_Individual,type_of_lead_sponsor_Industry,type_of_lead_sponsor_NIH,type_of_lead_sponsor_Other,type_of_lead_sponsor_US_Fed,type_of_lead_sponsor_University,gender_proper_Both,gender_proper_Female,gender_proper_Male,duration_in_days,trials_in_2_years,enrollment_num,os_activenotrecruiting,os_completed,os_enrollingbyinvitation,os_notyetrecruiting,os_random,os_recruiting,os_suspended,os_terminated,os_withdrawn,randomized_no,randomized_unknown,randomized_yes,masking_openlabel,masking_singleblind,masking_doubleblind,endpt_classifictn_safety,endpt_classifictn_efficacy,endpt_classifictn_pharmacokinetics,endpt_classifictn_effectiveness,armgrpclass_experimental,armgrpclass_active,armgrpclass_Placebo,armgrpclass_NoIntervention,armgrpclass_Sham,number_of_authors,number_of_princi_invest,authorclass_princiinvest,authorclass_studydir,authorclass_studychair,authorclass_sponsorinvestigator,completiondateclass_actual_completion,completiondateclass_anticipated_completion,Combination_Therapy,Comparison_Therapy,Monotherapy,purpose_None,purpose_Other,purpose_Prevention,purpose_Research,purpose_Supportive_Care,purpose_Treatment,intervention_Crossover_Assignment,intervention_Factorial_Assignment,intervention_None,intervention_Parallel_Assignment,intervention_Sequential_Assignment,intervention_Single_Group_Assignment,trial_activity_duration,phase_1,phase_2,phase_3,phase_4,phase_Others,is_rare_disease,KOL,condition_history_phase1,condition_history_phase1_com,condition_history_phase1_term,condition_history_phase2,condition_history_phase2_com,condition_history_phase2_term,condition_history_phase3,condition_history_phase3_com,condition_history_phase3_term,condition_history_phase4,condition_history_phase4_com,condition_history_phase4_term,condition_sponsor_history_phase1,condition_sponsor_history_phase1_com,condition_sponsor_history_phase1_term,condition_sponsor_history_phase2,condition_sponsor_history_phase2_com,condition_sponsor_history_phase2_term,condition_sponsor_history_phase3,condition_sponsor_history_phase3_com,condition_sponsor_history_phase3_term,condition_sponsor_history_phase4,condition_sponsor_history_phase4_com,condition_sponsor_history_phase4_term,count_of_drugs_exp,drug_condition_history_phase1,drug_condition_history_phase1_com,drug_condition_history_phase1_term,drug_condition_history_phase2,drug_condition_history_phase2_com,drug_condition_history_phase2_term,drug_condition_history_phase3,drug_condition_history_phase3_com,drug_condition_history_phase3_term,drug_condition_history_phase4,drug_condition_history_phase4_com,drug_condition_history_phase4_term,drug_history_phase1,drug_history_phase1_com,drug_history_phase1_term,drug_history_phase2,drug_history_phase2_com,drug_history_phase2_term,drug_history_phase3,drug_history_phase3_com,drug_history_phase3_term,drug_history_phase4,drug_history_phase4_com,drug_history_phase4_term,drug_sponsor_history_phase1,drug_sponsor_history_phase1_com,drug_sponsor_history_phase1_term,drug_sponsor_history_phase2,drug_sponsor_history_phase2_com,drug_sponsor_history_phase2_term,drug_sponsor_history_phase3,drug_sponsor_history_phase3_com,drug_sponsor_history_phase3_term,drug_sponsor_history_phase4,drug_sponsor_history_phase4_com,drug_sponsor_history_phase4_term,sponsor_history_phase1,sponsor_history_phase1_com,sponsor_history_phase1_term,sponsor_history_phase2,sponsor_history_phase2_com,sponsor_history_phase2_term,sponsor_history_phase3,sponsor_history_phase3_com,sponsor_history_phase3_term,sponsor_history_phase4,sponsor_history_phase4_com,sponsor_history_phase4_term,cong_drugcond_no_of_congresses,no_of_grants_condition,no_of_grants_drug,no_of_grants_drug_condition,no_of_patents_condition,no_of_patents_drug,no_of_patents_drug_condition,pub_cond_impact_factor,pub_cond_no_of_publications,pub_cond_sjr_impact_factor,pub_drug_impa

In [437]:
# len(model_features)
train_features_gbm.shape

(22273, 227)

In [438]:
# test_clinical_id = test_features['clinical_id'].tolist()+test_features_no_version['clinical_id'].tolist()
# test_version = test_features['version'].tolist() + test_features_no_version['version'].tolist()
# test_lastchanged_date = test_features['lastchanged_date'].tolist() + test_features_no_version['lastchanged_date'].tolist()
# test_overall_status_lower = test_features['overall_status_lower'].tolist() + test_features_no_version['overall_status_lower'].tolist()
# test_year = test_features['year'].tolist() + test_features_no_version['year'].tolist()
# len(test_year)
# test_features['index'] = 0
test_features_gbm = test_features_gbm[train_features_gbm.columns.tolist()]
print(test_features_gbm.shape)
# test_features_no_version.drop(columns = ['version','lastchanged_date','year'],inplace=True)
#print(test_features_no_version_gbm.shape)
#test_features_no_version_gbm = test_features_no_version_gbm[test_features_gbm.columns.tolist()]
#print(test_features_no_version_gbm.shape)
#test_features_gbm = pd.concat([test_features_gbm,test_features_no_version_gbm])
print(test_features_gbm.shape)
test_features_gbm.head(2)

(652, 227)
(652, 227)


,clinical_id,val,overall_status_lower,number_of_arms,num_of_countries,num_of_sites,number_of_collaborators,minimum_age_month,maximum_age_month,type_of_lead_sponsor_Hospital,type_of_lead_sponsor_Individual,type_of_lead_sponsor_Industry,type_of_lead_sponsor_NIH,type_of_lead_sponsor_Other,type_of_lead_sponsor_US_Fed,type_of_lead_sponsor_University,gender_proper_Both,gender_proper_Female,gender_proper_Male,duration_in_days,trials_in_2_years,enrollment_num,os_activenotrecruiting,os_completed,os_enrollingbyinvitation,os_notyetrecruiting,os_random,os_recruiting,os_suspended,os_terminated,os_withdrawn,randomized_no,randomized_unknown,randomized_yes,masking_openlabel,masking_singleblind,masking_doubleblind,endpt_classifictn_safety,endpt_classifictn_efficacy,endpt_classifictn_pharmacokinetics,endpt_classifictn_effectiveness,armgrpclass_experimental,armgrpclass_active,armgrpclass_Placebo,armgrpclass_NoIntervention,armgrpclass_Sham,number_of_authors,number_of_princi_invest,authorclass_princiinvest,authorclass_studydir,authorclass_studychair,authorclass_sponsorinvestigator,completiondateclass_actual_completion,completiondateclass_anticipated_completion,Combination_Therapy,Comparison_Therapy,Monotherapy,purpose_None,purpose_Other,purpose_Prevention,purpose_Research,purpose_Supportive_Care,purpose_Treatment,intervention_Crossover_Assignment,intervention_Factorial_Assignment,intervention_None,intervention_Parallel_Assignment,intervention_Sequential_Assignment,intervention_Single_Group_Assignment,trial_activity_duration,phase_1,phase_2,phase_3,phase_4,phase_Others,is_rare_disease,KOL,condition_history_phase1,condition_history_phase1_com,condition_history_phase1_term,condition_history_phase2,condition_history_phase2_com,condition_history_phase2_term,condition_history_phase3,condition_history_phase3_com,condition_history_phase3_term,condition_history_phase4,condition_history_phase4_com,condition_history_phase4_term,condition_sponsor_history_phase1,condition_sponsor_history_phase1_com,condition_sponsor_history_phase1_term,condition_sponsor_history_phase2,condition_sponsor_history_phase2_com,condition_sponsor_history_phase2_term,condition_sponsor_history_phase3,condition_sponsor_history_phase3_com,condition_sponsor_history_phase3_term,condition_sponsor_history_phase4,condition_sponsor_history_phase4_com,condition_sponsor_history_phase4_term,count_of_drugs_exp,drug_condition_history_phase1,drug_condition_history_phase1_com,drug_condition_history_phase1_term,drug_condition_history_phase2,drug_condition_history_phase2_com,drug_condition_history_phase2_term,drug_condition_history_phase3,drug_condition_history_phase3_com,drug_condition_history_phase3_term,drug_condition_history_phase4,drug_condition_history_phase4_com,drug_condition_history_phase4_term,drug_history_phase1,drug_history_phase1_com,drug_history_phase1_term,drug_history_phase2,drug_history_phase2_com,drug_history_phase2_term,drug_history_phase3,drug_history_phase3_com,drug_history_phase3_term,drug_history_phase4,drug_history_phase4_com,drug_history_phase4_term,drug_sponsor_history_phase1,drug_sponsor_history_phase1_com,drug_sponsor_history_phase1_term,drug_sponsor_history_phase2,drug_sponsor_history_phase2_com,drug_sponsor_history_phase2_term,drug_sponsor_history_phase3,drug_sponsor_history_phase3_com,drug_sponsor_history_phase3_term,drug_sponsor_history_phase4,drug_sponsor_history_phase4_com,drug_sponsor_history_phase4_term,sponsor_history_phase1,sponsor_history_phase1_com,sponsor_history_phase1_term,sponsor_history_phase2,sponsor_history_phase2_com,sponsor_history_phase2_term,sponsor_history_phase3,sponsor_history_phase3_com,sponsor_history_phase3_term,sponsor_history_phase4,sponsor_history_phase4_com,sponsor_history_phase4_term,cong_drugcond_no_of_congresses,no_of_grants_condition,no_of_grants_drug,no_of_grants_drug_condition,no_of_patents_condition,no_of_patents_drug,no_of_patents_drug_condition,pub_cond_impact_factor,pub_cond_no_of_publications,pub_cond_sjr_impact_factor,pub_drug_impa

In [439]:
train_labels_gbm=np.array(train_features_gbm['val'])
# remove = ["val","clinical_id",'overall_status_lower','primary_measure', 'os_activenotrecruiting',
# 'os_completed',
# 'os_recruiting',
# 'os_withdrawn',
# 'os_notyetrecruiting',
# 'os_random',
# 'os_suspended',
# 'os_terminated','DrugBank_Entry']
remove = ["val","clinical_id","overall_status_lower"]
train_features_gbm=np.array(train_features_gbm.drop(remove,axis=1))
#train_features=np.array(train_features.drop(["val","clinical_id"],axis=1))

test_features_gbm_val = test_features_gbm[test_features_gbm['val']!=2]
test_labels_gbm_val=np.array(test_features_gbm_val['val'])
test_features_gbm_val=np.array(test_features_gbm_val.drop(remove,axis=1))

test_labels_gbm=np.array(test_features_gbm['val'])
test_features_gbm=np.array(test_features_gbm.drop(remove,axis=1))
#test_features=np.array(test_features.drop(["val","clinical_id"],axis=1))

train_one_hot = to_categorical(train_labels_gbm)

print(train_features_gbm.shape,train_labels_gbm.shape, test_features_gbm.shape,test_labels_gbm.shape, test_features_gbm_val.shape,test_labels_gbm_val.shape)

(22273, 224) (22273,) (652, 224) (652,) (426, 224) (426,)


In [440]:
# Train GBM Model
gbm=GradientBoostingClassifier(learning_rate=0.1,n_estimators=1000, max_depth=6,verbose=1) #,max_features='auto',loss='deviance',subsample=1)
gbm.fit(train_features_gbm,train_labels_gbm)

      Iter       Train Loss   Remaining Time 
         1           1.2864           19.05m
         2           1.2254           17.13m
         3           1.1714           16.42m
         4           1.1278           16.03m
         5           1.0906           15.71m
         6           1.0586           15.49m
         7           1.0279           15.38m
         8           1.0025           15.39m
         9           0.9755           15.41m
        10           0.9505           15.53m
        20           0.8052           15.42m
        30           0.7229           15.15m
        40           0.6731           14.61m
        50           0.6329           14.27m
        60           0.6059           13.69m
        70           0.5790           13.34m
        80           0.5553           12.97m
        90           0.5392           12.39m
       100           0.5209           12.12m
       200           0.3955            9.59m
       300           0.3088            8.03m
       40

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=6,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1000,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=1, warm_start=False)

In [171]:
# filename_gbm = 'Data/Priyank_data/gbm_12mar.pkl'
# filename_gbm = 'Data/Priyank_data/gbm_15apr_2_5k_test_acatis.pkl'
filename_gbm = 'Data/Priyank_data/gbm_26feb.pkl'

In [314]:
# pickle.dump(gbm, open(filename_gbm, 'wb'))

In [172]:
gbm1 = pickle.load(open(filename_gbm, 'rb'))

In [441]:
probs_gbm = gbm.predict_proba(test_features_gbm_val)
predicted_gbm = gbm.predict(test_features_gbm_val)
print("AUC: ", roc_auc_score(test_labels_gbm_val, probs_gbm[:,-1]))
print ("accuracy is ",metrics.accuracy_score(test_labels_gbm_val, predicted_gbm))
print (metrics.confusion_matrix(test_labels_gbm_val, predicted_gbm))
print (metrics.classification_report(test_labels_gbm_val, predicted_gbm))

AUC:  0.5911016949152542
accuracy is  0.5492957746478874
[[201 153]
 [ 39  33]]
              precision    recall  f1-score   support

         0.0       0.84      0.57      0.68       354
         1.0       0.18      0.46      0.26        72

   micro avg       0.55      0.55      0.55       426
   macro avg       0.51      0.51      0.47       426
weighted avg       0.73      0.55      0.61       426



# Predictions for Acatis

In [442]:
probs_gbm = gbm.predict_proba(test_features_gbm)
predicted_gbm = gbm.predict(test_features_gbm)

In [175]:
# predicted_gbm[:10]

In [444]:
probs_gbm1 = [np.array(x[1]).tolist() for x in probs_gbm]
# probs_gbm1

df_preds_gbm = pd.DataFrame({'clinical_id':test_clinical_id,'version':test_version,'overall_status_lower':test_overall_status_lower,'lastchanged_date':test_lastchanged_date,'year':test_year,'probability_1':probs_gbm1,'val':test_labels_gbm,'prediction':np.array(list(predicted_gbm))})
print(df_preds_gbm.shape)
df_preds_gbm.head()

(652, 8)


,clinical_id,lastchanged_date,overall_status_lower,prediction,probability_1,val,version,year
0,NCT00545129,2009-07-22,recruiting,1.0,0.658408,0.0,37.0,2009
1,NCT00545129,2010-10-12,recruiting,1.0,0.624290,0.0,52.0,2010
2,NCT00545129,2010-08-19,recruiting,1.0,0.624290,0.0,51.0,2010
3,NCT00545129,2009-12-01,recruiting,1.0,0.663508,0.0,42.0,2009
4,NCT00545129,2009-07-02,recruiting,1.0,0.658408,0.0,36.0,2009


In [454]:
df_preds_gbm[df_preds_gbm['val'] == df_preds_gbm['prediction']].clinical_id.unique()#.sort_values(by = ['version'], ascending = False)

array(['NCT00545129', 'NCT00744471', 'NCT00784693', 'NCT00809354',
       'NCT00809783', 'NCT00863772', 'NCT00924664', 'NCT00960804',
       'NCT00985621', 'NCT00994890', 'NCT00999518', 'NCT01087203',
       'NCT01089725', 'NCT01127893', 'NCT01146561'], dtype=object)

In [488]:
df_preds_nn[df_preds_nn['clinical_id'] == 'NCT00809354'].shape

(36, 8)

In [1]:
! pwd

/data/users-workspace/anshul.jain/code/clinical_trial/phase_prediction


In [50]:
pd.concat([df_final2[(df_final2['clinical_id'] == 'NCT00744471') & (df_final2['version'] == 32)], df_final2[(df_final2['clinical_id'] == 'NCT00809354') & (df_final2['version'] == 35)],df_final2[(df_final2['clinical_id'] == 'NCT02528188') & (df_final2['version'] == 68)], df_final2[(df_final2['clinical_id'] == 'NCT02697773') & (df_final2['version'] == 38)]])[['clinical_id','enrollment_num']]

,clinical_id,enrollment_num
30808,NCT00744471,627.0
33248,NCT00809354,2720.0
105393,NCT02528188,3021.0
117875,NCT02697773,698.0


In [473]:
df_final1[(df_final1['clinical_id'] == 'NCT02528188') & (df_final1['version'] == 68)]

,Combination_Therapy,Comparison_Therapy,DrugBank_Entry,DrugGroup__approved,DrugGroup__experimental,DrugGroup__illicit,DrugGroup__investigational,DrugGroup__nutraceutical,DrugGroup__vet_approved,DrugGroup__withdrawn,Drugs_Normalized,KOL,Mapped_Drug,Medium_aerosol,Medium_capsule,Medium_injection,Medium_kit,Medium_lotion,Medium_other,Medium_patch,Medium_powder,Medium_solution,Medium_suspension,Medium_tablet,Monotherapy,TA_class_TA_cardiology,TA_class_TA_dermatology,TA_class_TA_endocrinology_metabolic,TA_class_TA_gastroenterology,TA_class_TA_genetic,TA_class_TA_hematology,TA_class_TA_immunology,TA_class_TA_infections,TA_class_TA_musculoskeletal,TA_class_TA_nephrology,TA_class_TA_neurology,TA_class_TA_obstetrics_gynaecology,TA_class_TA_oncology,TA_class_TA_ophthalmology,TA_class_TA_others,TA_class_TA_respiratory,TA_class_TA_urology,_id,approved_drug_count_by_sponsor,armgrpclass_NoIntervention,armgrpclass_Placebo,armgrpclass_Sham,armgrpclass_active,armgrpclass_experimental,aromatic_rings_count,authorclass_princiinvest,authorclass_sponsorinvestigator,authorclass_studychair,authorclass_studydir,biological_IC,biological_enzyme,biological_gpcr,biological_kinase,biological_nr,biological_transporter,biological_unspecified,clinical_id,completiondateclass_actual_completion,completiondateclass_anticipated_completion,condition,condition_history_phase1,condition_history_phase1_com,condition_history_phase1_term,condition_history_phase2,condition_history_phase2_com,condition_history_phase2_term,condition_history_phase3,condition_history_phase3_com,condition_history_phase3_term,condition_history_phase4,condition_history_phase4_com,condition_history_phase4_term,condition_sponsor_history_phase1,condition_sponsor_history_phase1_com,condition_sponsor_history_phase1_term,condition_sponsor_history_phase2,condition_sponsor_history_phase2_com,condition_sponsor_history_phase2_term,condition_sponsor_history_phase3,condition_sponsor_history_phase3_com,condition_sponsor_history_phase3_term,condition_sponsor_history_phase4,condition_sponsor_history_phase4_com,condition_sponsor_history_phase4_term,cong_drugcond_no_of_congresses,count_of_drugs_exp,created_at,created_at_ES,created_at_timestamp,diff_in_completiondate,drug_condition_history_phase1,drug_condition_history_phase1_com,drug_condition_history_phase1_term,drug_condition_history_phase2,drug_condition_history_phase2_com,drug_condition_history_phase2_term,drug_condition_history_phase3,drug_condition_history_phase3_com,drug_condition_history_phase3_term,drug_condition_history_phase4,drug_condition_history_phase4_com,drug_condition_history_phase4_term,drug_history_phase1,drug_history_phase1_com,drug_history_phase1_term,drug_history_phase2,drug_history_phase2_com,drug_history_phase2_term,drug_history_phase3,drug_history_phase3_com,drug_history_phase3_term,drug_history_phase4,drug_history_phase4_com,drug_history_phase4_term,drug_sponsor_history_phase1,drug_sponsor_history_phase1_com,drug_sponsor_history_phase1_term,drug_sponsor_history_phase2,drug_sponsor_history_phase2_com,drug_sponsor_history_phase2_term,drug_sponsor_history_phase3,drug_sponsor_history_phase3_com,drug_sponsor_history_phase3_term,drug_sponsor_history_phase4,drug_sponsor_history_phase4_com,drug_sponsor_history_phase4_term,duration_in_days,endpt_classifictn_effectiveness,endpt_classifictn_efficacy,endpt_classifictn_pharmacokinetics,endpt_classifictn_safety,enrollment_num,gender_proper_Both,gender_proper_Female,gender_proper_Male,heavy_atoms_count,hydrogen_bond_acceptor_count,hydrogen_bond_donor_count,id_info,if_all_drugs_for_some_indication,if_any_drug_prior_approved,indication_approved_for_all_drugs,intervention_Crossover_Assignment,intervention_Factorial_Assignment,intervention_None,intervention_Parallel_Assignment,intervention_Sequential_Assignment,intervention_Single_Group_Assignment,isTop20,is_rare_disease,lastchanged_date,logp_value,masking_doubleblind,masking_openlabel,masking_singleblind,maximum_age_month,minimum_age_month,molecular_weig

In [455]:
df_final[(df_final['clinical_id']=='NCT00863772')]#.clinical_id.nunique()

,clinical_id,val,overall_status_lower,number_of_arms,num_of_countries,num_of_sites,number_of_collaborators,minimum_age_month,maximum_age_month,type_of_lead_sponsor_Hospital,type_of_lead_sponsor_Individual,type_of_lead_sponsor_Industry,type_of_lead_sponsor_NIH,type_of_lead_sponsor_Other,type_of_lead_sponsor_US_Fed,type_of_lead_sponsor_University,gender_proper_Both,gender_proper_Female,gender_proper_Male,duration_in_days,trials_in_2_years,enrollment_num,os_activenotrecruiting,os_completed,os_enrollingbyinvitation,os_notyetrecruiting,os_random,os_recruiting,os_suspended,os_terminated,os_withdrawn,randomized_no,randomized_unknown,randomized_yes,masking_openlabel,masking_singleblind,masking_doubleblind,endpt_classifictn_safety,endpt_classifictn_efficacy,endpt_classifictn_pharmacokinetics,endpt_classifictn_effectiveness,armgrpclass_experimental,armgrpclass_active,armgrpclass_Placebo,armgrpclass_NoIntervention,armgrpclass_Sham,number_of_authors,number_of_princi_invest,authorclass_princiinvest,authorclass_studydir,authorclass_studychair,authorclass_sponsorinvestigator,completiondateclass_actual_completion,completiondateclass_anticipated_completion,Combination_Therapy,Comparison_Therapy,Monotherapy,purpose_None,purpose_Other,purpose_Prevention,purpose_Research,purpose_Supportive_Care,purpose_Treatment,intervention_Crossover_Assignment,intervention_Factorial_Assignment,intervention_None,intervention_Parallel_Assignment,intervention_Sequential_Assignment,intervention_Single_Group_Assignment,trial_activity_duration,phase_1,phase_2,phase_3,phase_4,phase_Others,is_rare_disease,KOL,count_of_drugs_exp,drug_condition_history_phase1,drug_condition_history_phase2,drug_condition_history_phase3,drug_condition_history_phase4,sponsor_history_phase1,sponsor_history_phase2,sponsor_history_phase3,sponsor_history_phase4,cong_drugcond_no_of_congresses,approved_drug_count_by_sponsor,diff_in_completiondate,if_all_drugs_for_some_indication,if_any_drug_prior_approved,indication_approved_for_all_drugs,no_of_drugs_approved_for_indication,variations_in_completiondate_using_versions,polar_surface_area,hydrogen_bond_acceptor_count,hydrogen_bond_donor_count,violations_in_octet_rule_count,heavy_atoms_count,logp_value,rotatable_bond_count,aromatic_rings_count,molecular_weight,no_of_metabolites_incr,no_of_metabolites_decr,no_of_targets,no_of_pathways,no_of_manufacturer,pharmacological_other,pharmacological_activator,pharmacological_agonist,pharmacological_antagonist,pharmacological_antibody,pharmacological_binder,pharmacological_inhibitor,biological_enzyme,biological_gpcr,biological_kinase,biological_nr,biological_transporter,biological_unspecified,biological_IC,type_biotech,state_gas,state_liquid,state_solid,status_of_drug_marketed,status_of_drug_non_marketed,status_of_drug_other,DrugGroup__approved,DrugGroup__experimental,DrugGroup__illicit,DrugGroup__investigational,DrugGroup__nutraceutical,DrugGroup__vet_approved,DrugGroup__withdrawn,Medium_aerosol,Medium_capsule,Medium_injection,Medium_kit,Medium_lotion,Medium_other,Medium_patch,Medium_powder,Medium_solution,Medium_suspension,Medium_tablet,isTop20,impact_factor,no_of_publications,no_of_grants,no_of_patents
35155,NCT00863772,0.0,terminated,3.0,1,63,0,216.0,1200.0,0.0,0.0,1,0.0,0.0,0.0,0.0,1,0,0.0,549.0,285,220.0,0,0,0.0,0,0.0,0,0.0,1.0,0.0,0.0,0.0,1,0,0,1,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0.0,0.0,1,0,0,0,0,0.0,1,0,0.0,0,1,0.0,0,102643200.0,0,0,1,0.0,0,0.0,0.0,1.0,66.0,175.0,288.0,249.0,436.0,1257.0,1604.0,753.0,0.0,171.0,31.0,0.0,0.0,0.0,21.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.09091,5272.0,2431.0,80292.0
35156,NCT00863772,0.0,suspended,3.0,2,67,0,216.0,1200.0,0.0,0.0,1,0.0,0.0,0.0,0.0,1,0,0.0,549.0,285,220.0,0,0,0.0,0,0.0,0,1.0,0.0,0.0,0.0,0.0,1,0,0,1,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0.0,0.0,1,0,0,0,0,0.0,1,0,0.0,0,1,0.0,0,1026

In [182]:
df_preds_gbm = df_preds_gbm[df_preds_gbm['version']!=0]
df_preds_gbm.shape

(63635, 8)

In [149]:
# df_preds_gbm[df_preds_gbm['clinical_id']=="NCT02978326"].sort_values(by=['version'])

In [183]:
# df_preds_gbm.to_csv('Data/Priyank_data/Acatis_2_5k_GBM_predictions_15April.csv',index=False)
df_preds_gbm.to_csv('Data/Priyank_data/Acatis_8k_GBM_predictions_16April_28feb_pickle.csv',index=False)

In [184]:
df_preds_gbm['clinical_id'].nunique()

7946

In [185]:
from collections import Counter
Counter(test_labels_gbm)

Counter({0.0: 7541, 1.0: 10801, 2.0: 53311})

In [186]:
# get_CTs_from_ES('NCT00126360')
len(listofOOS)

8018

In [ ]:
# # read data from ES for all the aforementioned trials and merge it with success/fail column 
# dffeat = get_CTs_from_ES(listofOOS)
# print (dffeat.shape)

In [ ]:
# dffeat['overall_status_lower'] = dffeat['overall_status'].str.lower()
# dffeat['overall_status_lower'].nunique()
# dffeat1= dffeat[['clinical_id', 'phase','overall_status_lower','verification_date','start_year','version']]
# dffeat1[:2]

# dffeat.rename(columns={'verification_date':'lastchanged_date'}, inplace=True)

# client = MongoClient('10.240.0.144',36018)
# client.admin.authenticate('yogesh.kumar', 'DEBdcydbedebhf', mechanism='SCRAM-SHA-1')
# DB_NAME = 'Drugs'
# db = client[DB_NAME]
# print(db)

# coll_on_the_fly_versions = db['clinical_trial_all_versions_non_drug_feature']
# dfversions = pd.DataFrame(list(coll_on_the_fly_versions.find()))
# print(dfversions.shape)

# dfversions = dfversions[dfversions['clinical_id'].isin(listofOOS)]
# dfversions = dfversions[['clinical_id','overall_status_lower','phase','start_year','version','changes','lastchanged_date']]
# dfversions[:2]

# allvers = pd.concat([dffeat, dfversions], axis = 0)
# allvers.drop_duplicates(subset = ['clinical_id','overall_status_lower'], inplace = True)
# allvers.shape

In [ ]:
# dfall_gbm = df_preds_gbm.merge(allvers, on = ['clinical_id','overall_status_lower'], how = 'left')

In [ ]:
# dfversions.shape

In [164]:
# predicted_nn[:1]

# NN+GBM (Don't run this, go to Ensemble predictions)

In [161]:
dfout=pd.concat([pd.DataFrame(probs_gbm),pd.DataFrame(predicted_gbm),pd.DataFrame(probs_nn),pd.DataFrame(predicted_nn),pd.DataFrame(test_labels_gbm)],axis=1)
print(dfout.shape)
dfout.columns = ['gbm_0','gbm_1','gbm_pred','nn_1','nn_pred','val']
dfout.head()

(71653, 6)


,gbm_0,gbm_1,gbm_pred,nn_1,nn_pred,val
0,0.065614,0.934386,1,0.446862,0,2
1,0.120822,0.879178,1,0.446862,0,2
2,0.120822,0.879178,1,0.446862,0,2
3,0.005158,0.994842,1,0.708223,1,2
4,0.935183,0.064817,0,0.575564,1,2


In [ ]:
# dfout[dfout['nn_pred']==dfout['val']].shape

In [165]:
dfout['nn_0'] = dfout['nn_1'].apply(lambda x : 1-x)
dfout = dfout[['gbm_0','gbm_1','gbm_pred','nn_0','nn_1','nn_pred','val']]
dfout.head(2)

,gbm_0,gbm_1,gbm_pred,nn_0,nn_1,nn_pred,val
0,0.065614,0.934386,1,0.553138,0.446862,0,2
1,0.120822,0.879178,1,0.553138,0.446862,0,2


In [ ]:
def return_merged_prob_0(x1,x2,x3):
    print(x1)
    print(type(x1))
    if x1==0:
        return (x2+x3)/2
    else:
        return (x2+(1-x3))/2
    
def return_merged_prob_1(x1,x2,x3):
    if x1==1:
        return (x2+x3)/2
    else:
        return (x2+(1-x3))/2

In [166]:

# dfout['prob_0'] = dfout[['nn_pred','gbm_0','nn_0_1']].apply(lambda x: return_merged_prob_0(x[0],x[1],x[2]))
# dfout['prob_1'] = dfout[['nn_pred','gbm_1','nn_0_1']].apply(lambda x: return_merged_prob_1(x[0],x[1],x[2]))
dfout['prob_0'] = (dfout['gbm_0']+dfout['nn_0'])/2
dfout['prob_1'] = (dfout['gbm_1']+dfout['nn_1'])/2
dfout['predicted'] = 0
dfout['predicted'][dfout['prob_1']>dfout['prob_0']] = 1
dfout[:2]

,gbm_0,gbm_1,gbm_pred,nn_0,nn_1,nn_pred,val,prob_0,prob_1,predicted
0,0.065614,0.934386,1,0.553138,0.446862,0,2,0.309376,0.690624,1
1,0.120822,0.879178,1,0.553138,0.446862,0,2,0.336980,0.663020,1


In [168]:
# print ("accuracy is ",metrics.accuracy_score(dfout['val'], dfout['predicted']))
# print ("AUC is ",metrics.roc_auc_score(dfout['val'], dfout['prob_1']))
# print ("confusion matrix: ")
# print (metrics.confusion_matrix(dfout['val'], dfout['predicted']))
# print ("precision and recall numbers: ")
# print (metrics.classification_report(dfout['val'], dfout['predicted']))

In [ ]:
# dfout['predicted']

In [169]:
dfout['val'].value_counts()

2    54506
1     9755
0     7392
Name: val, dtype: int64

In [170]:
Counter(test_labels_gbm)

Counter({2: 54506, 1: 9755, 0: 7392})

In [ ]:
# test_labels_gbm

In [173]:
#test_features_df = df_final[df_final['clinical_id'].isin(listofOOS)]
print(dfout.shape)

print(test_ncts.shape, test_ovc.shape, np.array(predicted_gbm).shape, test_labels.shape)

dfall = pd.DataFrame({'clinical_id':np.array(test_ncts), 'overall_status_lower':np.array(test_ovc),'Labels':dfout['val'], 'Predicted':dfout['predicted'], 'Probabilities':dfout['prob_1']})
#dfall = dfall.merge(dffeat[['clinical_id', 'overall_status']], left_on = ['NCT'], right_on = 'clinical_id', how = 'left')

In [174]:
dfall.head(2)

In [ ]:
# test_ncts

In [ ]:
dfall[dfall['clinical_id']=='NCT00055731']

In [ ]:
dfall.to_csv('Data/Priyank_data/Ensemble_NN_GBM_results_21feb.csv',index=False)

In [ ]:
dfall['clinical_id'].nunique()

In [ ]:
dfall1 = pd.DataFrame({'clinical_id':np.array(test_ncts), 'overall_status_lower':np.array(test_ovc),'Labels':dfout['val'],'gbm_0':dfout['gbm_0'],'gbm_1':dfout['gbm_1'],
                       'nn_0':dfout['nn_0'],'nn_1':dfout['nn_1'],'Predicted':dfout['predicted'], 'Probabilities':dfout['prob_1']})

In [ ]:
dfall1.head(10)

# Ensembe predictions

In [187]:
probs_gbm0 = [1-x for x in probs_gbm1]
# probs_gbm0[:5]

In [188]:
probs_nn0 = [1-x for x in probs_nn1]
# probs_nn0

In [189]:
df_preds_ensemble = pd.DataFrame({'clinical_id':test_clinical_id,'version':test_version,'overall_status_lower':test_overall_status_lower,
                                'lastchanged_date':test_lastchanged_date,'year':test_year,'val':test_labels_gbm,
                                'probability_gbm_0':probs_gbm0,'probability_gbm_1':probs_gbm1,'prediction_gbm':np.array(list(predicted_gbm)),
                                'probability_nn_0':probs_nn0,'probability_nn_1':probs_nn1,'prediction_nn':np.array(list(predicted_nn))})
print(df_preds_ensemble.shape)
df_preds_ensemble.head()

(71653, 12)


,clinical_id,lastchanged_date,overall_status_lower,prediction_gbm,prediction_nn,probability_gbm_0,probability_gbm_1,probability_nn_0,probability_nn_1,val,version,year
0,NCT00002663,2016-12-22,recruiting,1,1,0.065614,0.934386,0.372460,0.627540,2.0,51,2016
1,NCT00002663,2015-07-27,recruiting,1,1,0.120822,0.879178,0.372460,0.627540,2.0,50,2015
2,NCT00002663,2018-08-10,recruiting,1,1,0.120822,0.879178,0.372460,0.627540,2.0,54,2018
3,NCT00002663,0,recruiting,1,1,0.005158,0.994842,0.205999,0.794001,2.0,0,0
4,NCT00003645,2010-04-13,completed,0,1,0.989274,0.010726,0.270418,0.729582,2.0,11,2010


In [190]:
df_preds_ensemble['probability_ensemble_0'] = (df_preds_ensemble['probability_gbm_0']+df_preds_ensemble['probability_nn_0'])/2
df_preds_ensemble['probability_ensemble_1'] = (df_preds_ensemble['probability_gbm_1']+df_preds_ensemble['probability_nn_1'])/2
df_preds_ensemble['prediction_ensemble'] = 0
df_preds_ensemble['prediction_ensemble'][df_preds_ensemble['probability_ensemble_1']>df_preds_ensemble['probability_ensemble_0']] = 1
df_preds_ensemble[:2]

,clinical_id,lastchanged_date,overall_status_lower,prediction_gbm,prediction_nn,probability_gbm_0,probability_gbm_1,probability_nn_0,probability_nn_1,val,version,year,probability_ensemble_0,probability_ensemble_1,prediction_ensemble
0,NCT00002663,2016-12-22,recruiting,1,1,0.065614,0.934386,0.37246,0.62754,2.0,51,2016,0.219037,0.780963,1
1,NCT00002663,2015-07-27,recruiting,1,1,0.120822,0.879178,0.37246,0.62754,2.0,50,2015,0.246641,0.753359,1


In [191]:
df_preds_ensemble[df_preds_ensemble['clinical_id']=="NCT00378482"]

,clinical_id,lastchanged_date,overall_status_lower,prediction_gbm,prediction_nn,probability_gbm_0,probability_gbm_1,probability_nn_0,probability_nn_1,val,version,year,probability_ensemble_0,probability_ensemble_1,prediction_ensemble
2276,NCT00378482,2016-07-21,"active, not recruiting",0,1,0.950961,0.049039,0.445147,0.554853,2.0,85,2016,0.698054,0.301946,0
2277,NCT00378482,2016-11-15,"active, not recruiting",0,1,0.950961,0.049039,0.445147,0.554853,2.0,86,2016,0.698054,0.301946,0
2278,NCT00378482,2016-03-29,"active, not recruiting",0,1,0.950961,0.049039,0.445147,0.554853,2.0,84,2016,0.698054,0.301946,0
2279,NCT00378482,0,"active, not recruiting",1,1,0.091445,0.908555,0.445147,0.554853,2.0,0,0,0.268296,0.731704,1
2280,NCT00378482,2011-06-08,recruiting,0,0,0.959842,0.040158,0.570188,0.429812,2.0,65,2011,0.765015,0.234985,0
2281,NCT00378482,2018-11-19,"active, not recruiting",0,1,0.941637,0.058363,0.445147,0.554853,2.0,92,2018,0.693392,0.306608,0
2282,NCT00378482,2016-11-15,"active, not recruiting",0,1,0.950961,0.049039,0.445147,0.554853,2.0,86,2016,0.698054,0.301946,0


In [748]:
# df_preds_ensemble[df_preds_ensemble['clinical_id']=="NCT02978326"].sort_values(by=['version'])

In [593]:
# df_preds_ensemble['overall_status_lower'][(df_preds_ensemble['clinical_id']=="NCT02978326")&(df_preds_ensemble['version']==19)] = 'active, not recruiting'

In [ ]:
# df_preds_ensemble[df_preds_ensemble['clinical_id']=="NCT02978326"].sort_values(by=['version'])[['clinical']]

In [749]:
# df_preds_ensemble[df_preds_ensemble['clinical_id']=="NCT02978326"].shape

(0, 15)

In [596]:
# df_preds_ensemble[df_preds_ensemble['clinical_id']=="NCT02978326"].to_csv("Data/Priyank_data/Acatis_Ensemble_predictions_02april_NCT02978326.csv",index=False)

In [192]:
# df_preds_ensemble[df_preds_ensemble['overall_status_lower'].isnull()]['clinical_id'].nunique()
df_preds_ensemble[df_preds_ensemble['version']==0]['clinical_id'].nunique()

8018

In [193]:
df_preds_ensemble[~df_preds_ensemble['version'].isnull()]['clinical_id'].nunique()

8018

In [194]:
# df_preds_ensemble_1 = df_preds_ensemble[~df_preds_ensemble['version'].isnull()]
# df_preds_ensemble_1.shape

In [195]:
df_preds_ensemble = df_preds_ensemble[df_preds_ensemble['version']!=0]
df_preds_ensemble.shape

(63635, 15)

In [196]:
df_preds_ensemble['clinical_id'].nunique()

7946

In [197]:
df_preds_ensemble.to_csv("Data/Priyank_data/Acatis_8k_Ensemble_predictions_16April_28feb_pickle.csv",index=False)
# df_preds_ensemble.to_csv("Data/Priyank_data/Acatis_2_5k_Ensemble_predictions_15April.csv",index=False)

In [ ]:
# df_preds_ensemble_2 = df_preds_ensemble[df_preds_ensemble['version'].isnull()]
# df_preds_ensemble_2.shape

In [ ]:
# df_version_status = pd.DataFrame(list(db['clinical_trial_version_non_drug_feature_updated'].find()))
# print(df_version_status.shape)
# df_version_status.head(2)

In [ ]:
# df_version_2016 = pd.DataFrame(list(db['clinical_trial_all_versions_non_drug_feature'].find()))
# print(df_version_2016.shape)
# df_version_2016.head(2)

In [ ]:
# dfversion16 = df_version_2016[['clinical_id','overall_status_lower','phase','start_year','version','changes','lastchanged_date']]
# print(dfversion16.shape)
# dfversion16.head(2)

In [ ]:
# dfversion_status = df_version_status[['clinical_id','overall_status_lower','phase','start_year','version','changes','lastchanged_date']]
# print(dfversion_status.shape)
# dfversion_status.head(2)

In [ ]:
# dfversion16.isnull().sum()

In [ ]:
# pd.merge(df_preds_ensemble_1,dfversion16,on=['clinical_id','version','overall_status_lower','lastchanged_date'],how='left').head(1)

In [ ]:
# pd.merge(df_preds_ensemble_1,dfversion16,on=['clinical_id','version','overall_status_lower','lastchanged_date'],how='left').shape

In [ ]:
# df_preds_ensemble_2.drop(columns=['version','lastchanged_date','year'],inplace=True)

In [ ]:
# df_preds_ensemble_2.shape

In [ ]:
# df_preds_ensemble_2['clinical_id'].nunique()

In [ ]:
# df_preds_ensemble_2[df_preds_ensemble_2['clinical_id'].isin(dfversion_status['clinical_id'].unique().tolist())]['clinical_id'].nunique()

In [ ]:
# df_preds_ensemble_2.isnull().sum()

In [ ]:
# dfversion_status.drop_duplicates(subset=['clinical_id','overall_status_lower','changes']).shape

In [ ]:
# pd.merge(df_preds_ensemble_2,dfversion_status,on=['clinical_id','overall_status_lower'],how='left').head(1)

In [ ]:
# pd.merge(df_preds_ensemble_2,dfversion_status,on=['clinical_id','overall_status_lower'],how='left').drop_duplicates(subset=['clinical_id','overall_status_lower']).shape

In [ ]:
# df_preds_ensemble_2_final.isnull().sum()

In [ ]:
# df_preds_ensemble_1[df_preds_ensemble_1['clinical_id'].isin(dfversion16['clinical_id'].unique().tolist())].shape

In [ ]:
# df_preds_ensemble_2[df_preds_ensemble_2['clinical_id'].isin(dfversion_status['clinical_id'].unique().tolist())].shape

In [ ]:
# temp_not_found = df_preds_ensemble_2[df_preds_ensemble_2['clinical_id'].isin(dfversion_status['clinical_id'].unique().tolist())]
# temp_not_found[temp_not_found['clinical_id'].isin(dfversion16['clinical_id'].unique().tolist())].shape

In [ ]:
# df_preds_ensemble_1_final = pd.merge(df_preds_ensemble_1,dfversion16,on=['clinical_id','version','overall_status_lower','lastchanged_date'],how='left')
# print(df_preds_ensemble_1_final.shape)
# df_preds_ensemble_1_final.head(1)

In [ ]:
# df_preds_ensemble_2_final = pd.merge(df_preds_ensemble_2,dfversion_status,on=['clinical_id','overall_status_lower'],how='left').drop_duplicates(subset=['clinical_id','overall_status_lower'])
# print(df_preds_ensemble_2_final.shape)
# df_preds_ensemble_2_final['year'] = df_preds_ensemble_2_final['lastchanged_date'].apply(lambda x: str(x)[:4])
# df_preds_ensemble_2_final = df_preds_ensemble_2_final[df_preds_ensemble_1_final.columns.tolist()]
# df_preds_ensemble_2_final.head(1)

In [ ]:
# df_preds_ensemble_2_final[df_preds_ensemble_2_final['clinical_id']=='NCT00323856']

In [ ]:
# df_preds_ensemble_1_final[df_preds_ensemble_1_final['start_year'].isnull()]

In [ ]:
# df_preds_final = pd.concat([df_preds_ensemble_1_final,df_preds_ensemble_2_final])
# print(df_preds_final.shape)
# df_preds_final.head(1)

# Change labels for NCTs whose label was wrong

In [ ]:
# df_change_val = pd.read_csv('Data/change_val.csv')
# df_change_val.drop_duplicates(inplace=True)
# print(df_change_val.shape)
# df_change_val.head(2)

In [ ]:
# df_preds_final['val'].value_counts()

In [ ]:
# df_preds_final.isnull().sum()

In [ ]:
# df_preds_final[df_preds_final['clinical_id']=='NCT02367183']

In [ ]:
# df_preds_final['val'][df_preds_final['clinical_id'].isin(df_change_val['NCT'].unique().tolist())]=2

In [ ]:
# df_preds_final['val'].value_counts()

In [ ]:
# df_preds_final_temp = df_preds_final.copy()
# df_preds_final_temp.shape

In [ ]:
# df_preds_final_temp['clinical_id'].nunique()

In [ ]:
# df_preds_final_temp.dropna(inplace=True)

In [ ]:
# df_preds_final_temp['clinical_id'].nunique()

In [ ]:
# df_preds_final_temp.isnull().sum()

In [ ]:
# df_preds_final_temp[df_preds_final_temp['clinical_id']=='NCT01493414']

In [ ]:
# dfversion_status[dfversion_status['clinical_id']=='NCT01493414']

# get start year, phase and other info which is missing

In [752]:
df_preds_final = df_preds_ensemble.copy()

all_ncts = df_preds_final['clinical_id'].unique().tolist()
len(all_ncts)

2523

In [753]:
df_all_trials_info = get_CTs_from_ES(all_ncts)
print(df_all_trials_info.shape)
df_all_trials_info.head(2)

(2523, 107)


,abstract,acronym,age_congresses,age_publications,age_publications_congresses,arm_group,authors,authors_merged,clinical_id,completion_date,completion_date_source,condition,condition_mesh_terms,countries,countries_joined,created_at,data_source,detailed_description,duration,eligibility,end_date,endpoints_main,enriched_flag,enrollment,firstreceived_date,firstreceived_date_source,firstreceived_results_date,firstreceived_results_date_source,firstreceived_results_disposition_date,firstreceived_results_disposition_date_source,gender,gender_congresses,gender_publications,gender_publications_congresses,healthy_volunteers,id_info,innoplexus_id,intervention,intervention_mesh_terms,is_enriched,is_fda_regulated,keyword,language,last_updated,last_updated_dev,location,mapped_documents,maximum_age,maximum_age_congresses,maximum_age_publications,maximum_age_publications_congresses,meta_analysis_list,minimum_age,minimum_age_congresses,minimum_age_publications,minimum_age_publications_congresses,more_info,normalized_location,normalized_sites_joined,number_of_arms,official_title,other_sources,overall_contact,overall_contact_backup,overall_official,overall_status,overall_status_ctc,oversight_info,patient_num,patient_num_old,phase,primary_completion_date,primary_completion_date_source,primary_outcome,priority,public_title,race,race_publications,race_publications_congresses,races_joined,recruitment_status,reference,removed_countries,responsible_party,results_reference,secondary_outcome,sites_joined,source,source_url,sponsor_type,sponsors,start_date,start_date_source,start_year,study_design,study_design_ctc,study_design_tagged,study_design_tagged_congpub,study_docs,study_results,study_type,study_type_ctc,therapy_type,title,verification_date,verification_date_source,why_stopped
0,PRIMARY OBJECTIVES: Determination of safety an...,NaN,NaN,NaN,NaN,[{'arm_group_label': 'Dendritic Cells (DC) mal...,"[{'author_name': 'Svein Dueland', 'role': 'Pri...",[{'investigator_affiliation': 'Oslo University...,NCT01278940,"{'type': 'Actual', 'date': '2015-12-01'}","{'type': 'Actual', 'date': 'December 2015'}",[Malignant Melanoma],[Melanoma],NaN,NaN,1014940800,ClinicalTrials.gov,NaN,NaN,"{'healthy_volunteers': 'No', 'gender': 'Both',...",NaN,NaN,NaN,"{'number': '31', 'type': 'Actual'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'org_study_id': 'DC malignant melanoma', 'nct...",360aeb80ccba86ab8a29cc200d1f92e2e3f1a4a5570263...,[{'arm_group_label': ['Dendritic Cells (DC) ma...,[Vaccines],NaN,No,NaN,english,1549377339,1553608780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Phase I/II Trial of Vaccine Therapy With mRNA-...,NaN,NaN,NaN,"[{'role': 'Principal Investigator', 'last_name...",Completed,NaN,{'authority': ['Norway: Norwegian Medicines Ag...,NaN,NaN,Phase 1/Phase 2,"{'type': 'Actual', 'date': '2006-02-01'}","{'type': 'Actual', 'date': 'February 2006'}","[{'safety_issue': 'Yes', 'description': 'Bioch...",1,Trial of Vaccine Therapy With mRNA- Transfecte...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'investigator_full_name': 'Svein Dueland', 'i...","[{'PMID': '16710345', 'citation': 'Kyte JA, Mu...","[{'safety_issue': 'No', 'measure': 'Determine ...",NaN,Oslo University Hospital,https://clinicaltrials.gov/show/NCT01278940,NaN,"{'lead_sponsor': [{'countries': ['Norway'], 'a...",2002-03-01,March 2002,2002,"[{'type': 'Allocation', 'value': 'Non-Randomiz...",NaN,NaN,NaN,NaN,NaN,Interventional,NaN,NaN,NaN,2016-08-01,August 2016,NaN
1,GSK2800528 is being developed as a human anti-...,NaN,NaN,NaN,NaN,"[{'arm_group_label': 'GSK2800528 Arm', 'descri...",NaN,NaN,NCT01899755,"{'type': 'Actual', 'date': '2014-04-03'}","{'type': 'Actual', 'date': 'April 3, 2014'}",[Psoriasis],[Psoriasis],[United Kingdom],NaN,1373414400,ClinicalTrials.gov,NaN,NaN,{'healthy_volunteers': 'Accepts Healthy Volunt...,NaN,NaN,NaN,"{'number': '45', 'type': 'Actual'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'org_study_id': '116987', 'nct_id': 'NCT01899...",5ce19c81b2a60fa2751dd2df689abeb1a45dcabc66

In [754]:
# df_all_trials_info.isnull().sum()

In [755]:
dict_phase = {}
dict_start = {}
dict_lastchanged = {}
for index,rows in df_all_trials_info.iterrows():
    dict_phase[rows['clinical_id']] = rows['phase']
    dict_start[rows['clinical_id']] = rows['start_year']
    dict_lastchanged[rows['clinical_id']] = [str(rows['overall_status']).lower(),rows['verification_date']]

In [756]:
# dict_lastchanged

In [757]:
# df_preds_final_temp['phase'] = df_preds_final_temp['clinical_id'].apply(lambda x: dict_phase[x])
df_preds_final['phase'] = df_preds_final['clinical_id'].apply(lambda x: dict_phase[x])

In [758]:
# df_preds_final_temp['start_year'] = df_preds_final_temp['clinical_id'].apply(lambda x: dict_start[x])
df_preds_final['start_year'] = df_preds_final['clinical_id'].apply(lambda x: dict_start[x])

In [759]:
# for l in df_preds_final[df_preds_final['clinical_id']=='NCT00003645']['lastchanged_date']:
#     print(type(l))
df_preds_final[df_preds_final['clinical_id']=='NCT01370642']

,clinical_id,lastchanged_date,overall_status_lower,prediction_gbm,prediction_nn,probability_gbm_0,probability_gbm_1,probability_nn_0,probability_nn_1,val,version,year,probability_ensemble_0,probability_ensemble_1,prediction_ensemble,phase,start_year
10930,NCT01370642,2015-10-01,completed,1.0,1,0.018300,0.981700,0.100121,0.899879,1.0,15.0,2015,0.059211,0.940789,1,Phase 3,2011
10931,NCT01370642,2011-07-28,recruiting,1.0,1,0.108048,0.891952,0.267488,0.732512,1.0,3.0,2011,0.187768,0.812232,1,Phase 3,2011
10932,NCT01370642,2011-06-10,not yet recruiting,1.0,0,0.450905,0.549095,0.731916,0.268084,1.0,2.0,2011,0.591410,0.408590,0,Phase 3,2011
10933,NCT01370642,2011-10-31,recruiting,1.0,1,0.108048,0.891952,0.267488,0.732512,1.0,4.0,2011,0.187768,0.812232,1,Phase 3,2011
10934,NCT01370642,2011-11-23,recruiting,1.0,1,0.108048,0.891952,0.267488,0.732512,1.0,5.0,2011,0.187768,0.812232,1,Phase 3,2011
10935,NCT01370642,2011-12-02,recruiting,1.0,1,0.108048,0.891952,0.267488,0.732512,1.0,6.0,2011,0.187768,0.812232,1,Phase 3,2011
10936,NCT01370642,2012-02-29,recruiting,1.0,1,0.108048,0.891952,0.267488,0.732512,1.0,7.0,2012,0.187768,0.812232,1,Phase 3,2011
10937,NCT01370642,2012-04-02,"active, not recruiting",1.0,1,0.096308,0.903692,0.202163,0.797837,1.0,8.0,2012,0.149236,0.850764,1,Phase 3,2011
10938,NCT01370642,2012-05-24,"active, not recruiting",1.0,1,0.100704,0.899296,0.202714,0.797286,1.0,9.0,2012,0.151709,0.848291,1,Phase 3,2011
10939,NCT01370642,2018-09-21,completed,1.0,1,0.018300,0.981700,0.100121,0.899879,1.0,18.0,2018,0.059211,0.940789,1,Phase 3,2011


In [760]:
# def get_lastchaged_date(x1,x2,x3):
#     #print(x1,x2,x3)
#     if (isinstance(x2,str) and str(x2)=='') or pd.isnull(x2):
#         if x3==dict_lastchanged[x1][0]:
#             return dict_lastchanged[x1][1]
#         else:
#             return ''
#     else:
#         return x2
def get_lastchaged_date(x1,x2,x3):
    #print(x1,x2,x3)
    if (isinstance(x2,int) and int(x2)==0) or pd.isnull(x2):
        if x3==dict_lastchanged[x1][0]:
            return dict_lastchanged[x1][1]
        else:
            return ''
    else:
        return x2

In [761]:
# df_preds_final_temp.reset_index(drop=True,inplace=True)
df_preds_final.reset_index(drop=True,inplace=True)

In [762]:
# df_preds_final_temp['lastchanged_date'] = df_preds_final_temp[['clinical_id','lastchanged_date','overall_status_lower']].apply(lambda x: get_lastchaged_date(x['clinical_id'],x['lastchanged_date'],x['overall_status_lower']),axis=1)
df_preds_final['lastchanged_date'] = df_preds_final[['clinical_id','lastchanged_date','overall_status_lower']].apply(lambda x: get_lastchaged_date(x['clinical_id'],x['lastchanged_date'],x['overall_status_lower']),axis=1)

In [763]:
# df_preds_final_temp['year'] = df_preds_final_temp['lastchanged_date'].apply(lambda x: str(x)[:4])
df_preds_final['year'] = df_preds_final['lastchanged_date'].apply(lambda x: str(x)[:4])

In [764]:
# df_preds_final_temp[df_preds_final_temp['clinical_id']=='NCT00323856']
df_preds_final[df_preds_final['clinical_id']=='NCT01370642']

,clinical_id,lastchanged_date,overall_status_lower,prediction_gbm,prediction_nn,probability_gbm_0,probability_gbm_1,probability_nn_0,probability_nn_1,val,version,year,probability_ensemble_0,probability_ensemble_1,prediction_ensemble,phase,start_year
10930,NCT01370642,2015-10-01,completed,1.0,1,0.018300,0.981700,0.100121,0.899879,1.0,15.0,2015,0.059211,0.940789,1,Phase 3,2011
10931,NCT01370642,2011-07-28,recruiting,1.0,1,0.108048,0.891952,0.267488,0.732512,1.0,3.0,2011,0.187768,0.812232,1,Phase 3,2011
10932,NCT01370642,2011-06-10,not yet recruiting,1.0,0,0.450905,0.549095,0.731916,0.268084,1.0,2.0,2011,0.591410,0.408590,0,Phase 3,2011
10933,NCT01370642,2011-10-31,recruiting,1.0,1,0.108048,0.891952,0.267488,0.732512,1.0,4.0,2011,0.187768,0.812232,1,Phase 3,2011
10934,NCT01370642,2011-11-23,recruiting,1.0,1,0.108048,0.891952,0.267488,0.732512,1.0,5.0,2011,0.187768,0.812232,1,Phase 3,2011
10935,NCT01370642,2011-12-02,recruiting,1.0,1,0.108048,0.891952,0.267488,0.732512,1.0,6.0,2011,0.187768,0.812232,1,Phase 3,2011
10936,NCT01370642,2012-02-29,recruiting,1.0,1,0.108048,0.891952,0.267488,0.732512,1.0,7.0,2012,0.187768,0.812232,1,Phase 3,2011
10937,NCT01370642,2012-04-02,"active, not recruiting",1.0,1,0.096308,0.903692,0.202163,0.797837,1.0,8.0,2012,0.149236,0.850764,1,Phase 3,2011
10938,NCT01370642,2012-05-24,"active, not recruiting",1.0,1,0.100704,0.899296,0.202714,0.797286,1.0,9.0,2012,0.151709,0.848291,1,Phase 3,2011
10939,NCT01370642,2018-09-21,completed,1.0,1,0.018300,0.981700,0.100121,0.899879,1.0,18.0,2018,0.059211,0.940789,1,Phase 3,2011


In [765]:
get_CTs_from_ES('NCT01370642')

,abstract,arm_group,authors_merged,clinical_id,completion_date,condition,condition_mesh_terms,created_at,data_source,eligibility,endpoints_main,enriched_flag,enrollment,firstreceived_date,firstreceived_results_date,gender_publications,gender_publications_congresses,id_info,innoplexus_id,intervention,intervention_mesh_terms,is_enriched,language,last_updated,last_updated_dev,mapped_documents,number_of_arms,official_title,other_sources,overall_official,overall_status,oversight_info,patient_num,patient_num_old,phase,primary_completion_date,primary_outcome,priority,public_title,race_publications,race_publications_congresses,recruitment_status,removed_countries,responsible_party,results_reference,secondary_outcome,source,source_url,sponsors,start_date,start_year,study_design,study_design_tagged_congpub,study_docs,study_results,study_type,therapy_type,verification_date
0,The purpose of this study is to evaluate the s...,"[{'arm_group_label': 'Vaniprevir 12 Week Arm',...","[{'normalize_name': 'Norio Hayashi', 'new_auth...",NCT01370642,"{'type': 'Actual', 'date': '2014-03-01'}","[Hepatitis C, Chronic]","[Hepatitis, Hepatitis A, Hepatitis C, Hepatiti...",1306866600,ClinicalTrials.gov,"{'healthy_volunteers': 'No', 'gender': 'All', ...","[{'type': 'ctpub', 'tagged_list': ['control ar...","{'enroll': 1, 'study_design': 1, 'endpoints': 1}","{'number': '294', 'type': 'Actual'}",2011-06-08,2014-09-26,Both,Both,"{'org_study_id': '7009-043', 'nct_id': 'NCT013...",b571605597921653d3ac85926595a74ce49f51e661273d...,[{'arm_group_label': ['Vaniprevir 12 Week Arm'...,"[Ribavirin, Peginterferon alfa-2b]",1,english,1549043745,1553491683,"[{'doc_id': '26403160', 'method': 'context_bas...",3,"A Phase III Randomized, Placebo-controlled Stu...",[26403160],"[{'role': 'Study Director', 'last_name': 'Medi...",Completed,{'has_dmc': 'Yes'},294,294,Phase 3,"{'type': 'Actual', 'date': '2013-07-01'}",[{'time_frame': '24 weeks after 24 or 48 weeks...,1,Vaniprevir Administered With Pegylated-interfe...,{'inclusion': ['japanese']},{'inclusion': ['japanese']},Completed,[Japan],{'responsible_party_type': 'Sponsor'},"[{'PMID': '26403160', 'citation': 'Hayashi N, ...",[{'time_frame': '12 weeks after 24 or 48 weeks...,Merck Sharp & Dohme Corp.,https://clinicaltrials.gov/show/NCT01370642,{'lead_sponsor': [{'agency': 'Merck Sharp & Do...,2011-06-01,2011,"[{'type': 'allocation', 'value': 'Randomized'}...",[Comparative],[{'doc_url': 'http://www.merck.com/clinical-tr...,[{'reporting_groups': [{'title': 'Vaniprevir 1...,Interventional,Comparison Therapy,2017-03-01


In [766]:
# df_preds_final[df_preds_final['clinical_id']=='NCT00323856']
# df_preds_final['val'].value_counts()

In [767]:
df_preds_final.shape

(45324, 17)

In [768]:
# get_lastchaged_date('NCT00323856','','active, not recruiting')

In [769]:
df_preds_final[df_preds_final['year']==0].shape

(0, 17)

In [770]:
# df_preds_final.isnull().sum()
df_preds_final.drop_duplicates(subset=['clinical_id','version']).shape

(45324, 17)

In [771]:
df_preds_final[df_preds_final['val']!=2].shape

(13405, 17)

In [772]:
df_preds_final[(df_preds_final['val']!=2)&(df_preds_final['val']==df_preds_final['prediction_ensemble'])].shape

(11116, 17)

In [773]:
11116/13405

0.829242819843342

# Confusion matrix for Ensemble predictions

In [774]:
df_ensemble_temp = df_preds_final[df_preds_final['val']!=2]
print(df_ensemble_temp.shape)

(13405, 17)


In [775]:
probs_ensemble = df_ensemble_temp['probability_ensemble_1'].tolist() 
predicted_ensemble = df_ensemble_temp['prediction_ensemble']
print("AUC: ", roc_auc_score(df_ensemble_temp['val'], probs_ensemble))
print ("accuracy is ",metrics.accuracy_score(df_ensemble_temp['val'], predicted_ensemble))
print (metrics.confusion_matrix(df_ensemble_temp['val'], predicted_ensemble))
print (metrics.classification_report(df_ensemble_temp['val'], predicted_ensemble))

AUC:  0.8956154512738868
accuracy is  0.829242819843342
[[3983 1386]
 [ 903 7133]]
             precision    recall  f1-score   support

        0.0       0.82      0.74      0.78      5369
        1.0       0.84      0.89      0.86      8036

avg / total       0.83      0.83      0.83     13405



In [776]:
df_ensemble_temp['overall_status_lower'].value_counts()

recruiting                 6747
active, not recruiting     3260
completed                  1330
not yet recruiting          890
terminated                  478
suspended                   307
withdrawn                   264
enrolling by invitation     100
os_completed                 25
os_activenotrecruiting        2
os_recruiting                 1
                              1
Name: overall_status_lower, dtype: int64

In [777]:
df_ensemble_temp['clinical_id'].nunique()
# 9166/615

615

In [778]:
df_preds_final['val'].value_counts()

2.0    31919
1.0     8036
0.0     5369
Name: val, dtype: int64

In [779]:
df_preds_final['overall_status_lower'].value_counts()

recruiting                  25790
active, not recruiting       8423
completed                    4109
not yet recruiting           3570
os_completed                  762
suspended                     653
terminated                    622
os_recruiting                 516
withdrawn                     303
enrolling by invitation       283
os_activenotrecruiting        173
os_notyetrecruiting            64
os_terminated                  28
os_withdrawn                   12
os_enrollingbyinvitation       12
os_suspended                    3
                                1
Name: overall_status_lower, dtype: int64

In [780]:
df_preds_ensemble[df_preds_ensemble['overall_status_lower']=='os_suspended']['clinical_id'].nunique()

3

In [289]:
762+516+173+64+28+12+12+3

1570

In [457]:
2523-1570

953

# Save to csv

In [783]:
df_preds_final.shape

(45324, 17)

In [786]:
45324-2523

42801

In [781]:
df_preds_final.to_csv('Data/Priyank_data/Acatis_2_5k_Ensemble_predictions_03april.csv',index=False)

In [782]:
df_preds_final['lastchanged_date'].isnull().sum()

0

# Don't run below

In [337]:
# # df_preds_final.shape
# df_preds_final_1 = pd.read_csv('Data/Priyank_data/Acatis_8k_Ensemble_predictions_26feb.csv')
# print(df_preds_final_1.shape)
# df_preds_final_1.head(1)

(71653, 17)


,clinical_id,version,overall_status_lower,lastchanged_date,year,val,probability_gbm_0,probability_gbm_1,prediction_gbm,probability_nn_0,probability_nn_1,prediction_nn,probability_ensemble_0,probability_ensemble_1,prediction_ensemble,phase,start_year
0,NCT00002663,51,recruiting,2016-12-22,2016.0,2,0.065614,0.934386,1,0.553138,0.446862,0,0.309376,0.690624,1,Phase 2,1995.0


In [338]:
# df_preds_final.isnull().sum()
# df_preds_final_1.drop_duplicates(subset=['clinical_id','version']).shape

(65067, 17)

In [342]:
# df_preds_final_1['merged'] = df_preds_final_1[['clinical_id','version']].apply(lambda x:(x['clinical_id']+"_"+str(x['version'])),axis=1)
# df_preds_final_1.head(1)

,clinical_id,version,overall_status_lower,lastchanged_date,year,val,probability_gbm_0,probability_gbm_1,prediction_gbm,probability_nn_0,probability_nn_1,prediction_nn,probability_ensemble_0,probability_ensemble_1,prediction_ensemble,phase,start_year,merged
0,NCT00002663,51,recruiting,2016-12-22,2016.0,2,0.065614,0.934386,1,0.553138,0.446862,0,0.309376,0.690624,1,Phase 2,1995.0,NCT00002663_51


In [344]:
# df_preds_final_1['merged'].value_counts().head(30)

In [ ]:
# ncts_temp = df_preds_final[df_preds_final['version'].isnull()]['clinical_id'].tolist()
# dfversion16[dfversion16['clinical_id'].isin(ncts_temp)].shape#.isin(dfversion16.clinical_id.unique().tolist())

In [ ]:
# ncts_temp[0]

In [ ]:
# dfversion16[dfversion16['clinical_id']=='NCT00323856']

In [ ]:
# len([x for x in ncts_temp if x in dfversion_status.clinical_id.unique().tolist()])

In [ ]:
# dfversion_status[dfversion_status['clinical_id']=='NCT00323856']

# Testing

In [261]:
get_CTs_from_ES('NCT01370642')

,abstract,arm_group,authors_merged,clinical_id,completion_date,condition,condition_mesh_terms,created_at,data_source,eligibility,endpoints_main,enriched_flag,enrollment,firstreceived_date,firstreceived_results_date,gender_publications,gender_publications_congresses,id_info,innoplexus_id,intervention,intervention_mesh_terms,is_enriched,language,last_updated,last_updated_dev,mapped_documents,number_of_arms,official_title,other_sources,overall_official,overall_status,oversight_info,patient_num,patient_num_old,phase,primary_completion_date,primary_outcome,priority,public_title,race_publications,race_publications_congresses,recruitment_status,removed_countries,responsible_party,results_reference,secondary_outcome,source,source_url,sponsors,start_date,start_year,study_design,study_design_tagged_congpub,study_docs,study_results,study_type,therapy_type,verification_date
0,The purpose of this study is to evaluate the s...,"[{'arm_group_label': 'Vaniprevir 12 Week Arm',...","[{'normalize_name': 'Norio Hayashi', 'new_auth...",NCT01370642,"{'type': 'Actual', 'date': '2014-03-01'}","[Hepatitis C, Chronic]","[Hepatitis, Hepatitis A, Hepatitis C, Hepatiti...",1306866600,ClinicalTrials.gov,"{'healthy_volunteers': 'No', 'gender': 'All', ...","[{'type': 'ctpub', 'tagged_list': ['control ar...","{'enroll': 1, 'study_design': 1, 'endpoints': 1}","{'number': '294', 'type': 'Actual'}",2011-06-08,2014-09-26,Both,Both,"{'org_study_id': '7009-043', 'nct_id': 'NCT013...",b571605597921653d3ac85926595a74ce49f51e661273d...,[{'arm_group_label': ['Vaniprevir 12 Week Arm'...,"[Ribavirin, Peginterferon alfa-2b]",1,english,1549043745,1553491683,"[{'doc_id': '26403160', 'method': 'context_bas...",3,"A Phase III Randomized, Placebo-controlled Stu...",[26403160],"[{'role': 'Study Director', 'last_name': 'Medi...",Completed,{'has_dmc': 'Yes'},294,294,Phase 3,"{'type': 'Actual', 'date': '2013-07-01'}",[{'time_frame': '24 weeks after 24 or 48 weeks...,1,Vaniprevir Administered With Pegylated-interfe...,{'inclusion': ['japanese']},{'inclusion': ['japanese']},Completed,[Japan],{'responsible_party_type': 'Sponsor'},"[{'PMID': '26403160', 'citation': 'Hayashi N, ...",[{'time_frame': '12 weeks after 24 or 48 weeks...,Merck Sharp & Dohme Corp.,https://clinicaltrials.gov/show/NCT01370642,{'lead_sponsor': [{'agency': 'Merck Sharp & Do...,2011-06-01,2011,"[{'type': 'allocation', 'value': 'Randomized'}...",[Comparative],[{'doc_url': 'http://www.merck.com/clinical-tr...,[{'reporting_groups': [{'title': 'Vaniprevir 1...,Interventional,Comparison Therapy,2017-03-01


In [353]:
df_final[df_final["clinical_id"] == "NCT01370642"]["Combination_Therapy"][0:1].values[0]

0.0

In [358]:
# df_final[df_final['clinical_id']=='NCT01370642'][['molecular_weight','Comparison_Therapy','Combination_Therapy','Monotherapy','maximum_age_month','enrollment_num','num_of_sites','number_of_arms','minimum_age_month','number_of_authors','version','val']]
df_final[df_final['clinical_id']=='NCT01370642'].describe()

,Combination_Therapy,Comparison_Therapy,DrugGroup__approved,DrugGroup__experimental,DrugGroup__illicit,DrugGroup__investigational,DrugGroup__nutraceutical,DrugGroup__vet_approved,DrugGroup__withdrawn,KOL,Medium_aerosol,Medium_capsule,Medium_injection,Medium_kit,Medium_lotion,Medium_other,Medium_patch,Medium_powder,Medium_solution,Medium_suspension,Medium_tablet,Monotherapy,TA_class_TA_cardiology,TA_class_TA_dermatology,TA_class_TA_endocrinology_metabolic,TA_class_TA_gastroenterology,TA_class_TA_genetic,TA_class_TA_hematology,TA_class_TA_immunology,TA_class_TA_infections,TA_class_TA_musculoskeletal,TA_class_TA_nephrology,TA_class_TA_neurology,TA_class_TA_obstetrics_gynaecology,TA_class_TA_oncology,TA_class_TA_ophthalmology,TA_class_TA_others,TA_class_TA_respiratory,TA_class_TA_urology,approved_drug_count_by_sponsor,armgrpclass_NoIntervention,armgrpclass_Placebo,armgrpclass_Sham,armgrpclass_active,armgrpclass_experimental,aromatic_rings_count,authorclass_princiinvest,authorclass_sponsorinvestigator,authorclass_studychair,authorclass_studydir,biological_IC,biological_enzyme,biological_gpcr,biological_kinase,biological_nr,biological_transporter,biological_unspecified,completiondateclass_actual_completion,completiondateclass_anticipated_completion,condition_history_phase1,condition_history_phase1_com,condition_history_phase1_term,condition_history_phase2,condition_history_phase2_com,condition_history_phase2_term,condition_history_phase3,condition_history_phase3_com,condition_history_phase3_term,condition_history_phase4,condition_history_phase4_com,condition_history_phase4_term,condition_sponsor_history_phase1,condition_sponsor_history_phase1_com,condition_sponsor_history_phase1_term,condition_sponsor_history_phase2,condition_sponsor_history_phase2_com,condition_sponsor_history_phase2_term,condition_sponsor_history_phase3,condition_sponsor_history_phase3_com,condition_sponsor_history_phase3_term,condition_sponsor_history_phase4,condition_sponsor_history_phase4_com,condition_sponsor_history_phase4_term,cong_drugcond_no_of_congresses,count_of_drugs_exp,created_at_ES,diff_in_completiondate,drug_condition_history_phase1,drug_condition_history_phase1_com,drug_condition_history_phase1_term,drug_condition_history_phase2,drug_condition_history_phase2_com,drug_condition_history_phase2_term,drug_condition_history_phase3,drug_condition_history_phase3_com,drug_condition_history_phase3_term,drug_condition_history_phase4,drug_condition_history_phase4_com,drug_condition_history_phase4_term,drug_history_phase1,drug_history_phase1_com,drug_history_phase1_term,drug_history_phase2,drug_history_phase2_com,drug_history_phase2_term,drug_history_phase3,drug_history_phase3_com,drug_history_phase3_term,drug_history_phase4,drug_history_phase4_com,drug_history_phase4_term,drug_sponsor_history_phase1,drug_sponsor_history_phase1_com,drug_sponsor_history_phase1_term,drug_sponsor_history_phase2,drug_sponsor_history_phase2_com,drug_sponsor_history_phase2_term,drug_sponsor_history_phase3,drug_sponsor_history_phase3_com,drug_sponsor_history_phase3_term,drug_sponsor_history_phase4,drug_sponsor_history_phase4_com,drug_sponsor_history_phase4_term,duration_in_days,endpt_classifictn_effectiveness,endpt_classifictn_efficacy,endpt_classifictn_pharmacokinetics,endpt_classifictn_safety,enrollment_num,gender_proper_Both,gender_proper_Female,gender_proper_Male,heavy_atoms_count,hydrogen_bond_acceptor_count,hydrogen_bond_donor_count,if_all_drugs_for_some_indication,if_any_drug_prior_approved,indication_approved_for_all_drugs,intervention_Crossover_Assignment,intervention_Factorial_Assignment,intervention_None,intervention_Parallel_Assignment,intervention_Sequential_Assignment,intervention_Single_Group_Assignment,isTop20,is_rare_disease,logp_value,masking_doubleblind,masking_openlabel,masking_singleblind,maximum_age_month,minimum_age_month,molecular_weight,no_of_drugs_approved_for_indication,no_of_grants_condition,no_of_grants_drug,no_of_grants_drug_condition,no_of_manufacturer,

In [253]:
df_final[df_final['clinical_id']=='NCT00000113'].describe()
df_final[df_final['clinical_id']=='NCT03708861'][['molecular_weight','Comparison_Therapy','minimum_age_month','version','val']]

,molecular_weight,Comparison_Therapy,minimum_age_month,version,val
137058,513.67,0.0,216.0,1.0,0.0
137059,513.67,0.0,216.0,0.0,0.0


In [248]:
df_final[df_final['val']!=2]['clinical_id'].unique()[-30:]

array(['NCT03693573', 'NCT03694262', 'NCT03696160', 'NCT03696264',
       'NCT03698292', 'NCT03700320', 'NCT03701282', 'NCT03703791',
       'NCT03703882', 'NCT03708861', 'NCT03712345', 'NCT03715114',
       'NCT03720418', 'NCT03720470', 'NCT03722498', 'NCT03724240',
       'NCT03725852', 'NCT03738865', 'NCT03739866', 'NCT03740373',
       'NCT03742713', 'NCT03745131', 'NCT03748199', 'NCT03748953',
       'NCT03755791', 'NCT03758885', 'NCT03759379', 'NCT03781375',
       'NCT03786302', 'NCT03841344'], dtype=object)

In [222]:
df_final['clinical_id'].unique()[-20:]
# df_final['clinical_id'].unique()[:50]

array(['NCT03771040', 'NCT03774290', 'NCT03777059', 'NCT03781375',
       'NCT03786302', 'NCT03798236', 'NCT03802344', 'NCT03811678',
       'NCT03816267', 'NCT03819218', 'NCT03833362', 'NCT03836664',
       'NCT03841344', 'NCT03848806', 'NCT03858686', 'NCT03864042',
       'NCT03874832', 'NCT03880357', 'NCT03880734', 'NCT03885154'],
      dtype=object)

In [351]:
df_temp = df_preds_final_1[df_preds_final_1['val']!=2].drop_duplicates(subset=['clinical_id','version'])
df_temp.shape

(15945, 18)

In [352]:
df_temp[df_temp['val']==df_temp['prediction_ensemble']].shape

(12094, 18)

In [353]:
print ("accuracy is ",metrics.accuracy_score(df_temp['val'], df_temp['prediction_ensemble']))
print ("AUC is ",metrics.roc_auc_score(df_temp['val'], df_temp['prediction_ensemble']))
print ("confusion matrix: ")
print (metrics.confusion_matrix(df_temp['val'], df_temp['prediction_ensemble']))
print ("precision and recall numbers: ")
print (metrics.classification_report(df_temp['val'], df_temp['prediction_ensemble']))

accuracy is  0.7584822828472876
AUC is  0.7698709825751269
confusion matrix: 
[[5763 1038]
 [2813 6331]]
precision and recall numbers: 
              precision    recall  f1-score   support

           0       0.67      0.85      0.75      6801
           1       0.86      0.69      0.77      9144

   micro avg       0.76      0.76      0.76     15945
   macro avg       0.77      0.77      0.76     15945
weighted avg       0.78      0.76      0.76     15945



In [ ]:
8073/10700

In [ ]:
df_preds_ensemble_1_final[(df_preds_ensemble_1_final['year']=='2016')]['clinical_id'].nunique()

In [ ]:
df_preds_ensemble_1_final['clinical_id'].nunique()

In [ ]:
479+599

In [ ]:
df_preds_ensemble_1_final[df_preds_ensemble_1_final['year']!='2016']['clinical_id'].nunique()

In [ ]:
ncts_has_2016_version = df_preds_final[df_preds_final['year']=='2016']['clinical_id'].unique().tolist()
len(ncts_has_2016_version)

In [ ]:
df_preds_final[~df_preds_final['clinical_id'].isin(ncts_has_2016_version)]['clinical_id'].nunique()

# Calculate Accuracy

In [187]:
df_temp = df_preds_ensemble[df_preds_ensemble['val']!=2]
df_temp.shape

(17147, 15)

In [188]:
df_temp[df_temp['prediction_gbm']==df_temp['val']].shape

(12787, 15)

In [189]:
df_temp[df_temp['prediction_nn']==df_temp['val']].shape

(12587, 15)

In [190]:
df_temp[df_temp['prediction_ensemble']==df_temp['val']].shape

(13033, 15)

In [191]:
13033/17147

0.7600746486265819

In [192]:
12587/17147

0.7340642678019479

In [193]:
12787/17147

0.7457281157053712

In [ ]:
get_CTs_from_ES('NCT00323856')

In [ ]:
listofOOS[:15]

In [ ]:
df_all_vers = get_CTs_from_ES(listofOOS[:15])

In [ ]:
print(df_all_vers.shape)
# df_all_vers

In [ ]:
df_all_vers[df_all_vers['clinical_id']=='NCT02641392']

In [295]:
df_preds_ensemble.head(2)

,clinical_id,version,overall_status_lower,lastchanged_date,year,val,probability_gbm_0,probability_gbm_1,prediction_gbm,probability_nn_0,probability_nn_1,prediction_nn,probability_ensemble_0,probability_ensemble_1,prediction_ensemble
0,NCT00002663,51,recruiting,2016-12-22,2016,2,0.065614,0.934386,1,0.553138,0.446862,0,0.309376,0.690624,1
1,NCT00002663,50,recruiting,2015-07-27,2015,2,0.120822,0.879178,1,0.553138,0.446862,0,0.336980,0.663020,1


# Prepare final file for Acatis

In [225]:
df_acatis_final = pd.read_csv('Data/Priyank_data/Acatis_8k_Ensemble_predictions_26feb.csv')
df_acatis_final.head(2)

,clinical_id,version,overall_status_lower,lastchanged_date,year,val,probability_gbm_0,probability_gbm_1,prediction_gbm,probability_nn_0,probability_nn_1,prediction_nn,probability_ensemble_0,probability_ensemble_1,prediction_ensemble,phase,start_year
0,NCT00002663,51,recruiting,2016-12-22,2016.0,2,0.065614,0.934386,1,0.553138,0.446862,0,0.309376,0.690624,1,Phase 2,1995.0
1,NCT00002663,50,recruiting,2015-07-27,2015.0,2,0.120822,0.879178,1,0.553138,0.446862,0,0.336980,0.663020,1,Phase 2,1995.0


In [226]:
df_acatis_final.columns.tolist()

['clinical_id',
 'version',
 'overall_status_lower',
 'lastchanged_date',
 'year',
 'val',
 'probability_gbm_0',
 'probability_gbm_1',
 'prediction_gbm',
 'probability_nn_0',
 'probability_nn_1',
 'prediction_nn',
 'probability_ensemble_0',
 'probability_ensemble_1',
 'prediction_ensemble',
 'phase',
 'start_year']

In [227]:
df_acatis_final = df_acatis_final[['clinical_id',
 'version',
 'overall_status_lower',
 'lastchanged_date',
 'year',
 'val',
 'probability_ensemble_0',
 'probability_ensemble_1',
 'prediction_ensemble',
 'phase',
 'start_year']]
df_acatis_final.head(2)

,clinical_id,version,overall_status_lower,lastchanged_date,year,val,probability_ensemble_0,probability_ensemble_1,prediction_ensemble,phase,start_year
0,NCT00002663,51,recruiting,2016-12-22,2016.0,2,0.309376,0.690624,1,Phase 2,1995.0
1,NCT00002663,50,recruiting,2015-07-27,2015.0,2,0.336980,0.663020,1,Phase 2,1995.0


In [228]:
df_ticker = pd.read_csv('Data/Collab_Sponsor_Ticker_Acatis - ticker_fuzzy_results.csv')
print(df_ticker.shape)
df_ticker.head(2)

(9615, 5)


,clinical_id,sponsor,sponsor_lower,new_sponsor,ticker
0,NCT00193856,Abbott,abbott,abbott,ABT
1,NCT00598897,Abbott,abbott,abbott,ABT


In [ ]:
# [df_acatis_final["clinical_id"] == "NCT00193856"]

In [236]:
def return_ticker_sponsor(x):
    ticker=[]
    sponsor=[]
    try:
        ticker = list(set(df_ticker[df_ticker["clinical_id"] == x].ticker))
        sponsor = list(set(df_ticker[df_ticker["clinical_id"] == x].sponsor))
    except:
        pass
    return ticker,sponsor

In [237]:
df_acatis_final['ticker'] = np.nan
df_acatis_final['sponsor'] = np.nan

In [239]:
dict_ticker = dict()
dict_sponsor = dict()
ncts_unique = df_acatis_final['clinical_id'].unique().tolist()
len(ncts_unique)

8018

In [241]:
for cid in ncts_unique:
    ticker_out,sponsor_out = return_ticker_sponsor(cid)
    dict_ticker[cid] = ticker_out
    dict_sponsor[cid] = sponsor_out

In [243]:
df_acatis_final['ticker'] = df_acatis_final['clinical_id'].apply(lambda x: dict_ticker[x])

In [244]:
df_acatis_final['sponsor'] = df_acatis_final['clinical_id'].apply(lambda x: dict_sponsor[x])

In [247]:
df_acatis_final.head(2)

,clinical_id,version,overall_status_lower,lastchanged_date,year,val,probability_ensemble_0,probability_ensemble_1,prediction_ensemble,phase,start_year,ticker,sponsor
0,NCT00002663,51,recruiting,2016-12-22,2016.0,2,0.309376,0.690624,1,Phase 2,1995.0,[ATRA],[Atara Biotherapeutics]
1,NCT00002663,50,recruiting,2015-07-27,2015.0,2,0.336980,0.663020,1,Phase 2,1995.0,[ATRA],[Atara Biotherapeutics]


In [248]:
df_drug_condi_news = pd.read_csv('Data/all_acatis_test_trials (1)_all_info.csv')
print(df_drug_condi_news.shape)
df_drug_condi_news.head(2)

(8690, 23)


,Unnamed: 0,Experimental_drugs,_id,application_number,clinical_id,condition,created_at,created_at_ES,created_at_timestamp,id_info,news_date,news_link,other_id,overall_status,phase,publication_id,recruitment_status,source,source_url,start_date,sub_source_1,sub_source_2,val
0,0,['JCAR015'],5c61408654527c78e9ce055a,NaN,NCT02973191,['Precursor Cell Lymphoblastic Leukemia-Lympho...,2019-02-13,1482192000,1.550038e+09,"{'org_study_id': 'JCAR015-ALL-001', 'nct_id': ...",21/06/2016,https://www.biospace.com/article/kite-pharma-a...,NaN,withdrawn,phase 1/2,NaN,NaN,"suspended, terminated, withdrawn",https://clinicaltrials.gov/show/NCT02973191,2016-12-20,Sus_Terminated_withdrawn_23jan,acatis,0
1,1,NaN,5c61408654527c78e9ce055d,NaN,NCT02795416,['Psoriasis'],2019-02-13,1464739200,1.550038e+09,"{'org_study_id': 'CAIN457ATR01', 'nct_id': 'NC...",NaN,NaN,NaN,withdrawn,phase 3,NaN,NaN,"suspended, terminated, withdrawn",https://clinicaltrials.gov/show/NCT02795416,2016-06-01,Sus_Terminated_withdrawn_23jan,acatis,0


In [249]:
df_drug_condi_news = df_drug_condi_news[['clinical_id','Experimental_drugs','condition','news_link','news_date']]
print(df_drug_condi_news.shape)
df_drug_condi_news.head(2)

(8690, 5)


,clinical_id,Experimental_drugs,condition,news_link,news_date
0,NCT02973191,['JCAR015'],['Precursor Cell Lymphoblastic Leukemia-Lympho...,https://www.biospace.com/article/kite-pharma-a...,21/06/2016
1,NCT02795416,NaN,['Psoriasis'],NaN,NaN


In [282]:
df_acatis_final_merged = pd.merge(df_acatis_final,df_drug_condi_news,on='clinical_id',how='left')
print(df_acatis_final_merged.shape)

(71653, 17)


# Merge Reason for changes in version

In [284]:
df_version_2016 = pd.DataFrame(list(db['clinical_trial_all_versions_non_drug_feature'].find()))
print(df_version_2016.shape)
df_version_2016.head(2)

(66697, 204)


,Combination Therapy,Combination_Therapy,Comparison Therapy,Comparison_Therapy,Monotherapy,USFDA_regulated_device,USFDA_regulated_drug,_id,abstract,acronym,age_congresses,age_publications,age_publications_congresses,age_range_months,arm_group,armgrpclass_NoIntervention,armgrpclass_Placebo,armgrpclass_Sham,armgrpclass_active,armgrpclass_experimental,author_norm_flag,authorclass_princiinvest,authorclass_sponsorinvestigator,authorclass_studychair,authorclass_studydir,authors,authors_merged,biospec_descr,biospec_retention,biotagger_terms,changes,citations,clinical_id,completion_date,completion_date_source,completiondateclass_actual_completion,completiondateclass_anticipated_completion,condition,condition_mesh_terms,countries,countries_joined,created_at,created_at_timestamp,data_monitoring,data_source,detailed_description,diff_in_completiondate,duration,duration_in_days,eligibility,end_date,endpoints_main,endpt_classifictn_effectiveness,endpt_classifictn_efficacy,endpt_classifictn_pharmacokinetics,endpt_classifictn_safety,enriched_flag,enrollment,enrollment_is_actual,enrollment_num,first_posted,first_submitted_that_met_qc_criteria,firstreceived_date,firstreceived_date_source,firstreceived_results_date,firstreceived_results_date_source,firstreceived_results_disposition_date,firstreceived_results_disposition_date_source,gender,gender_based,gender_congresses,gender_proper,gender_proper_Both,gender_proper_Female,gender_proper_Male,gender_publications,gender_publications_congresses,get_pref_indication_name_part,healthy_volunteers,id_info,innoplexus_id,intervention,intervention_mesh_terms,is_actual_duration,is_affiliation_norm,is_enriched,is_fda_regulated,is_normalized_v2,is_rare_disease,keyword,language,last_update_dev,last_update_posted,last_updated,last_updated_dev,lastchanged_date,links,location,mapped_documents,masking_doubleblind,masking_openlabel,masking_singleblind,maximum_age,maximum_age_congresses,maximum_age_month,maximum_age_publications,maximum_age_publications_congresses,meta_analysis_list,minimum_age,minimum_age_congresses,minimum_age_month,minimum_age_publications,minimum_age_publications_congresses,more_info,new_author_norm_flag,normalized_location,normalized_sites_joined,num_of_countries,num_of_sites,number_of_arms,number_of_authors,number_of_collaborators,number_of_groups,number_of_princi_invest,official_title,original_overall_status,os_activenotrecruiting,os_completed,os_enrollingbyinvitation,os_notyetrecruiting,os_random,os_recruiting,os_suspended,os_terminated,os_withdrawn,other_sources,overall_contact,overall_contact_backup,overall_official,overall_status,overall_status_ctc,overall_status_lower,oversight_info,patient_num,patient_num_old,phase,plan_to_share_ipd,pref_indication_name,primary_completion_date,primary_completion_date_source,primary_outcome,primary_purpose,priority,public_title,race,race_publications,race_publications_congresses,races_publications_congresses_joined,races_publications_joined,randomized_no,randomized_or_not,randomized_unknown,randomized_yes,recruitment_status,reference,registry_name,removed_countries,responsible_party,results_reference,secondary_outcome,sites_joined,source,source_url,sponsor_type,sponsors,start_date,start_date_source,start_year,study_design,study_design_ctc,study_design_tagged,study_design_tagged_congpub,study_docs,study_results,study_type,study_type_ctc,target_duration,therapy_flag,therapy_type,title,trials_in_2_years,type_of_lead_sponsor,type_of_lead_sponsor_Industry,type_of_lead_sponsor_NIH,type_of_lead_sponsor_Nan,type_of_lead_sponsor_Other,type_of_lead_sponsor_US_Fed,update_label,update_label_ts,variations_in_completiondate_using_versions,verification_date,verification_date_source,version,why_stopped
0,NaN,0.0,NaN,NaN,1,,,5c6f63a454527c2166c90f36,This is a randomized study comparing the use o...,NaN,NaN,NaN,NaN,"{'maximum_age_month': 1200.0, 'minimum_age_mon...",[],0,0,0,0,0,1.0,4.0,0.0,0.0,0.0,[{'new_author_id': '03545c87cf2f4ae185f9162b48...,[{'source': {'asset_class

In [285]:
df_version_status = pd.DataFrame(list(db['clinical_trial_version_non_drug_feature_updated'].find()))
print(df_version_status.shape)
df_version_status.head(2)

(48003, 203)


,Combination_Therapy,Comparison_Therapy,Monotherapy,USFDA_regulated_device,USFDA_regulated_drug,_id,abstract,acronym,age_congresses,age_publications,age_publications_congresses,age_range_months,arm_group,armgrpclass_NoIntervention,armgrpclass_Placebo,armgrpclass_Sham,armgrpclass_active,armgrpclass_experimental,author_norm_flag,authorclass_princiinvest,authorclass_sponsorinvestigator,authorclass_studychair,authorclass_studydir,authors,authors_merged,biospec_descr,biospec_retention,biotagger_terms,changes,citations,clinical_id,completion_date,completion_date_source,completiondateclass_actual_completion,completiondateclass_anticipated_completion,condition,condition_mesh_terms,countries,countries_joined,created_at,created_at_timestamp,data_monitoring,data_source,detailed_description,diff_in_completiondate,duration,duration_in_days,eligibility,end_date,endpoints_main,endpt_classifictn_effectiveness,endpt_classifictn_efficacy,endpt_classifictn_pharmacokinetics,endpt_classifictn_safety,enriched_flag,enrollment,enrollment_is_actual,enrollment_num,first_posted,first_submitted_that_met_qc_criteria,firstreceived_date,firstreceived_date_source,firstreceived_results_date,firstreceived_results_date_source,firstreceived_results_disposition_date,firstreceived_results_disposition_date_source,gender,gender_based,gender_congresses,gender_proper,gender_proper_Both,gender_proper_Female,gender_proper_Male,gender_publications,gender_publications_congresses,get_pref_indication_name_part,healthy_volunteers,id_info,innoplexus_id,intervention,intervention_mesh_terms,is_actual_duration,is_affiliation_norm,is_enriched,is_fda_regulated,is_normalized_v2,is_rare_disease,keyword,language,last_update_dev,last_update_posted,last_updated,last_updated_dev,lastchanged_date,links,location,mapped_documents,mapped_pmid_list,masking_doubleblind,masking_openlabel,masking_singleblind,maximum_age,maximum_age_congresses,maximum_age_month,maximum_age_publications,maximum_age_publications_congresses,meta_analysis_list,minimum_age,minimum_age_congresses,minimum_age_month,minimum_age_publications,minimum_age_publications_congresses,more_info,new_author_norm_flag,normalized_location,normalized_sites_joined,num_of_countries,num_of_sites,number_of_arms,number_of_authors,number_of_collaborators,number_of_groups,number_of_princi_invest,official_title,original_overall_status,os_activenotrecruiting,os_completed,os_enrollingbyinvitation,os_notyetrecruiting,os_random,os_recruiting,os_suspended,os_withdrawn,other_sources,overall_contact,overall_contact_backup,overall_official,overall_status,overall_status_ctc,overall_status_lower,oversight_info,patient_num,patient_num_old,phase,plan_to_share_ipd,pref_indication_name,primary_completion_date,primary_completion_date_source,primary_outcome,primary_purpose,priority,public_title,race,race_publications,race_publications_congresses,races_publications_congresses_joined,races_publications_joined,randomized_no,randomized_or_not,randomized_unknown,randomized_yes,recruitment_start_date,recruitment_status,reference,registry_name,removed_countries,responsible_party,results_reference,secondary_outcome,sites_joined,source,source_url,sponsor_type,sponsors,start_date,start_date_source,start_year,study_design,study_design_ctc,study_design_tagged,study_design_tagged_congpub,study_docs,study_results,study_type,study_type_ctc,target_duration,therapy_flag,therapy_type,title,trials_in_2_years,type_of_lead_sponsor,type_of_lead_sponsor_Industry,type_of_lead_sponsor_NIH,type_of_lead_sponsor_Nan,type_of_lead_sponsor_Other,type_of_lead_sponsor_US_Fed,update_label,update_label_ts,variations_in_completiondate_using_versions,verification_date,verification_date_source,version,why_stopped
0,0,0,0,,,5c63c92954527c369fa75af4,The purpose of this study is to determine the ...,None,None,None,None,"{'maximum_age_month': 1200.0, 'minimum_age_mon...",[],0,0,0,0,0,NaN,0,0,0,0,[{'new_author_id': '402095fcc93442a6aa148b7275...,"[{'affiliation': 'CHU de Rouen', 'role': 'Prin...",No

In [286]:
df_allversions = pd.concat([df_version_2016,df_version_status])
print(df_allversions.shape)
df_all_trials_infoversions.drop_duplicates(subset=['clinical_id','version'],indf_all_trials_infoTrue)
print(df_allversions.shape)
df_allversions.head(1)

(114700, 206)
(98100, 206)


,Combination Therapy,Combination_Therapy,Comparison Therapy,Comparison_Therapy,Monotherapy,USFDA_regulated_device,USFDA_regulated_drug,_id,abstract,acronym,age_congresses,age_publications,age_publications_congresses,age_range_months,arm_group,armgrpclass_NoIntervention,armgrpclass_Placebo,armgrpclass_Sham,armgrpclass_active,armgrpclass_experimental,author_norm_flag,authorclass_princiinvest,authorclass_sponsorinvestigator,authorclass_studychair,authorclass_studydir,authors,authors_merged,biospec_descr,biospec_retention,biotagger_terms,changes,citations,clinical_id,completion_date,completion_date_source,completiondateclass_actual_completion,completiondateclass_anticipated_completion,condition,condition_mesh_terms,countries,countries_joined,created_at,created_at_timestamp,data_monitoring,data_source,detailed_description,diff_in_completiondate,duration,duration_in_days,eligibility,end_date,endpoints_main,endpt_classifictn_effectiveness,endpt_classifictn_efficacy,endpt_classifictn_pharmacokinetics,endpt_classifictn_safety,enriched_flag,enrollment,enrollment_is_actual,enrollment_num,first_posted,first_submitted_that_met_qc_criteria,firstreceived_date,firstreceived_date_source,firstreceived_results_date,firstreceived_results_date_source,firstreceived_results_disposition_date,firstreceived_results_disposition_date_source,gender,gender_based,gender_congresses,gender_proper,gender_proper_Both,gender_proper_Female,gender_proper_Male,gender_publications,gender_publications_congresses,get_pref_indication_name_part,healthy_volunteers,id_info,innoplexus_id,intervention,intervention_mesh_terms,is_actual_duration,is_affiliation_norm,is_enriched,is_fda_regulated,is_normalized_v2,is_rare_disease,keyword,language,last_update_dev,last_update_posted,last_updated,last_updated_dev,lastchanged_date,links,location,mapped_documents,mapped_pmid_list,masking_doubleblind,masking_openlabel,masking_singleblind,maximum_age,maximum_age_congresses,maximum_age_month,maximum_age_publications,maximum_age_publications_congresses,meta_analysis_list,minimum_age,minimum_age_congresses,minimum_age_month,minimum_age_publications,minimum_age_publications_congresses,more_info,new_author_norm_flag,normalized_location,normalized_sites_joined,num_of_countries,num_of_sites,number_of_arms,number_of_authors,number_of_collaborators,number_of_groups,number_of_princi_invest,official_title,original_overall_status,os_activenotrecruiting,os_completed,os_enrollingbyinvitation,os_notyetrecruiting,os_random,os_recruiting,os_suspended,os_terminated,os_withdrawn,other_sources,overall_contact,overall_contact_backup,overall_official,overall_status,overall_status_ctc,overall_status_lower,oversight_info,patient_num,patient_num_old,phase,plan_to_share_ipd,pref_indication_name,primary_completion_date,primary_completion_date_source,primary_outcome,primary_purpose,priority,public_title,race,race_publications,race_publications_congresses,races_publications_congresses_joined,races_publications_joined,randomized_no,randomized_or_not,randomized_unknown,randomized_yes,recruitment_start_date,recruitment_status,reference,registry_name,removed_countries,responsible_party,results_reference,secondary_outcome,sites_joined,source,source_url,sponsor_type,sponsors,start_date,start_date_source,start_year,study_design,study_design_ctc,study_design_tagged,study_design_tagged_congpub,study_docs,study_results,study_type,study_type_ctc,target_duration,therapy_flag,therapy_type,title,trials_in_2_years,type_of_lead_sponsor,type_of_lead_sponsor_Industry,type_of_lead_sponsor_NIH,type_of_lead_sponsor_Nan,type_of_lead_sponsor_Other,type_of_lead_sponsor_US_Fed,update_label,update_label_ts,variations_in_completiondate_using_versions,verification_date,verification_date_source,version,why_stopped
0,NaN,0.0,NaN,NaN,1,,,5c6f63a454527c2166c90f36,This is a randomized study comparing the use o...,NaN,NaN,NaN,NaN,"{'maximum_age_month': 1200.0, 'minimum_age_mon...",[],0,0,0,0,0,1.0,4.0,0.0,0.0,0.0,[{'new_author_id': '03545c87cf2f4ae

In [287]:
df_allversions_1 = df_allversions[['clinical_id','version','changes']]
print(df_allversions_1.shape)
df_allversions_1.head()

(98100, 3)


,clinical_id,version,changes
0,NCT00126360,2,"Recruitment Status, Outcome Measures, Eligibil..."
1,NCT00126360,3,Study Status
2,NCT00126360,1,Nothing (earliest Version on record)
3,NCT00295646,7,"Sponsor/Collaborators, Study Status, Study Ide..."
4,NCT00295646,14,Arms and Interventions and Study Status


In [290]:
# df_allversions_1.isnull().sum()
df_allversions_1.to_excel('Data/Acatis_Version_change_reasons.xlsx',index=False)

In [ ]:
# pd.merge(df_acatis_final_merged)

In [283]:
df_acatis_final_merged.head(2)

,clinical_id,version,overall_status_lower,lastchanged_date,year,val,probability_ensemble_0,probability_ensemble_1,prediction_ensemble,phase,start_year,ticker,sponsor,Experimental_drugs,condition,news_link,news_date
0,NCT00002663,51,recruiting,2016-12-22,2016.0,2,0.309376,0.690624,1,Phase 2,1995.0,[ATRA],[Atara Biotherapeutics],['allogeneic Epstein Barr virus specific cytot...,"['EBV-induced Lymphomas', 'EBV-associated Mali...",NaN,NaN
1,NCT00002663,50,recruiting,2015-07-27,2015.0,2,0.336980,0.663020,1,Phase 2,1995.0,[ATRA],[Atara Biotherapeutics],['allogeneic Epstein Barr virus specific cytot...,"['EBV-induced Lymphomas', 'EBV-associated Mali...",NaN,NaN


In [252]:
df_acatis_final_merged.rename(columns={'overall_status_lower':'overall_status','lastchanged_date':'version_change_date',
                                       'year':'version_change_year','probability_ensemble_0':'probability_0',
                                       'probability_ensemble_1':'probability_1','prediction_ensemble':'prediction',
                                       'Experimental_drugs':'Drugs','condition':'Disease'},inplace=True)

In [254]:
del df_acatis_final_merged['val']

In [255]:
df_acatis_final_merged.head(2)

,clinical_id,version,overall_status,version_change_date,version_change_year,probability_0,probability_1,prediction,phase,start_year,ticker,sponsor,Drugs,Disease,news_link,news_date
0,NCT00002663,51,recruiting,2016-12-22,2016.0,0.309376,0.690624,1,Phase 2,1995.0,[ATRA],[Atara Biotherapeutics],['allogeneic Epstein Barr virus specific cytot...,"['EBV-induced Lymphomas', 'EBV-associated Mali...",NaN,NaN
1,NCT00002663,50,recruiting,2015-07-27,2015.0,0.336980,0.663020,1,Phase 2,1995.0,[ATRA],[Atara Biotherapeutics],['allogeneic Epstein Barr virus specific cytot...,"['EBV-induced Lymphomas', 'EBV-associated Mali...",NaN,NaN


In [256]:
df_acatis_final_merged.columns.tolist()

['clinical_id',
 'version',
 'overall_status',
 'version_change_date',
 'version_change_year',
 'probability_0',
 'probability_1',
 'prediction',
 'phase',
 'start_year',
 'ticker',
 'sponsor',
 'Drugs',
 'Disease',
 'news_link',
 'news_date']

In [257]:
df_acatis_final_merged = df_acatis_final_merged[['clinical_id',
 'version',
 'overall_status',
 'version_change_date',
 'version_change_year',
 'phase',
 'start_year',
 'ticker',
 'sponsor',
 'Drugs',
 'Disease',
 'news_link',
 'news_date','probability_0',
 'probability_1',
 'prediction']]
print(df_acatis_final_merged.shape)
df_acatis_final_merged.head(2)

(71653, 16)


,clinical_id,version,overall_status,version_change_date,version_change_year,phase,start_year,ticker,sponsor,Drugs,Disease,news_link,news_date,probability_0,probability_1,prediction
0,NCT00002663,51,recruiting,2016-12-22,2016.0,Phase 2,1995.0,[ATRA],[Atara Biotherapeutics],['allogeneic Epstein Barr virus specific cytot...,"['EBV-induced Lymphomas', 'EBV-associated Mali...",NaN,NaN,0.309376,0.690624,1
1,NCT00002663,50,recruiting,2015-07-27,2015.0,Phase 2,1995.0,[ATRA],[Atara Biotherapeutics],['allogeneic Epstein Barr virus specific cytot...,"['EBV-induced Lymphomas', 'EBV-associated Mali...",NaN,NaN,0.336980,0.663020,1


In [259]:
df_acatis_final_merged.head(3)

,clinical_id,version,overall_status,version_change_date,version_change_year,phase,start_year,ticker,sponsor,Drugs,Disease,news_link,news_date,probability_0,probability_1,prediction
0,NCT00002663,51,recruiting,2016-12-22,2016.0,Phase 2,1995.0,[ATRA],[Atara Biotherapeutics],['allogeneic Epstein Barr virus specific cytot...,"['EBV-induced Lymphomas', 'EBV-associated Mali...",NaN,NaN,0.309376,0.690624,1
1,NCT00002663,50,recruiting,2015-07-27,2015.0,Phase 2,1995.0,[ATRA],[Atara Biotherapeutics],['allogeneic Epstein Barr virus specific cytot...,"['EBV-induced Lymphomas', 'EBV-associated Mali...",NaN,NaN,0.336980,0.663020,1
2,NCT00002663,54,recruiting,2018-08-10,2018.0,Phase 2,1995.0,[ATRA],[Atara Biotherapeutics],['allogeneic Epstein Barr virus specific cytot...,"['EBV-induced Lymphomas', 'EBV-associated Mali...",NaN,NaN,0.336980,0.663020,1


In [260]:
df_acatis_final_merged.sort_values(by=['clinical_id','version'],inplace=True)

In [281]:
# df_acatis_final_merged.head(30)

In [262]:
df_acatis_final_merged['clinical_id'].nunique()

8018

In [263]:
# df_acatis_final_merged.to_excel('Data/Acatis_8k_predictions.xlsx',index=False)

In [270]:
df_acatis_2016_verchange = df_acatis_final_merged[df_acatis_final_merged['version_change_year']==2016]
df_acatis_2016_verchange.shape

(39222, 16)

In [271]:
df_acatis_verchange = df_acatis_final_merged[df_acatis_final_merged['version_change_year']!=2016]
df_acatis_verchange.shape

(32431, 16)

In [274]:
# df_acatis_verchange.head(3)

In [276]:
df_acatis_verchange = df_acatis_verchange.drop_duplicates(subset=['clinical_id','overall_status'])
df_acatis_verchange.shape

(15027, 16)

In [277]:
df_acatis_final_merged_1 = pd.concat([df_acatis_2016_verchange,df_acatis_verchange])
print(df_acatis_final_merged_1.shape)
df_acatis_final_merged_1.head(2)

(54249, 16)


,clinical_id,version,overall_status,version_change_date,version_change_year,phase,start_year,ticker,sponsor,Drugs,Disease,news_link,news_date,probability_0,probability_1,prediction
0,NCT00002663,51,recruiting,2016-12-22,2016.0,Phase 2,1995.0,[ATRA],[Atara Biotherapeutics],['allogeneic Epstein Barr virus specific cytot...,"['EBV-induced Lymphomas', 'EBV-associated Mali...",NaN,NaN,0.309376,0.690624,1
5,NCT00003645,19,"active, not recruiting",2016-07-13,2016.0,Phase 3,1999.0,[AZN],[AstraZeneca],"['Flutamide', 'Leuprolide Acetate']",['Prostate Cancer'],NaN,NaN,0.814184,0.185816,0


In [279]:
df_acatis_final_merged_1.sort_values(by=['clinical_id','version'],inplace=True)

# Drop extra versions

In [294]:
c=-1
df_acatis_drop_extra = pd.DataFrame()
for index,rows in df_acatis_final_merged.iterrows():
    print(rows)
    break
    c+=1
    if c==0 or rows['version_change_date']==2016:
        df_acatis_drop_extra.append(rows)
    else:
        if (df_acatis_final_merged['overall_status'][c-1]!=rows['overall_status']) and (df_acatis_final_merged['clinical_id'][c-1]==rows['clinical_id']):
            df_acatis_drop_extra.append(rows)
c

clinical_id                                                     NCT00002663
version                                                                  51
overall_status_lower                                             recruiting
lastchanged_date                                                 2016-12-22
year                                                                   2016
val                                                                       2
probability_ensemble_0                                             0.309376
probability_ensemble_1                                             0.690624
prediction_ensemble                                                       1
phase                                                               Phase 2
start_year                                                             1995
ticker                                                               [ATRA]
sponsor                                             [Atara Biotherapeutics]
Experimental

-1

# Feature Importance

In [787]:
train_features_gbm_list = df_final_gbm.columns.tolist()
# remove = ["val","clinical_id",'overall_status_lower','primary_measure', 'os_activenotrecruiting',
# 'os_completed',
# 'os_recruiting',
# 'os_withdrawn',
# 'os_notyetrecruiting',
# 'os_random',
# 'os_suspended',
# 'os_terminated','DrugBank_Entry']
# remove = ["val","clinical_id","overall_status_lower","primary_measure","intervention_model","DrugBank_Entry",
#           "trial_activity_duration","duration_in_days"]
len(train_features_gbm_list)

268

In [788]:
len(model_features)

245

In [789]:
len(gbm1.feature_importances_)

242

In [790]:
remove = ["val","clinical_id","overall_status_lower"]
train_features_gbm_list = [i for i in train_features_gbm_list if i in model_features and i not in remove]

len(train_features_gbm_list)

242

In [791]:
feature_imp = pd.DataFrame(gbm1.feature_importances_,index=[train_features_gbm_list]).sort_values(by=[0],ascending=False)

In [792]:
feature_imp

,0
condition_history_phase3_com,0.036072
no_of_manufacturer,0.029775
no_of_grants_drug,0.028157
Medium_solution,0.027176
masking_singleblind,0.025679
Medium_patch,0.023555
no_of_metabolites_decr,0.022337
no_of_grants_drug_condition,0.021872
Medium_powder,0.021849
molecular_weight,0.021847


In [312]:
df_acatis_final_merged.drop_duplicates(subset=['clinical_id','version']).shape

(65067, 17)

In [322]:
from eli5.permutation_importance import get_score_importances

In [324]:
# perm = PermutationImportance(model_nn, random_state=1).fit([trainnum, traincat, trainovl] , train_labels)
def score(X, y):
    probs_nn = model_nn.predict(X) 
    predicted_nn = [1 if i >= 0.5 else 0 for i in probs_nn]
    return metrics.accuracy_score(y, predicted_nn)

base_score, score_decreases = get_score_importances(score, [trainnum, traincat, trainovl], train_labels)

AttributeError: 'list' object has no attribute 'shape'

In [333]:
df_temp_check = pd.read_excel('Data/Acatis_8k_predictions.xlsx')
print(df_temp_check.shape)
df_temp_check.head(2)

(71653, 16)


,clinical_id,version,overall_status,version_change_date,version_change_year,phase,start_year,ticker,sponsor,Drugs,Disease,news_link,news_date,probability_0,probability_1,prediction
0,NCT00002663,0,recruiting,2018-08-01,2018.0,Phase 2,1995.0,['ATRA'],['Atara Biotherapeutics'],['allogeneic Epstein Barr virus specific cytot...,"['EBV-induced Lymphomas', 'EBV-associated Mali...",NaN,NaN,0.148468,0.851532,1
1,NCT00002663,50,recruiting,2015-07-27,2015.0,Phase 2,1995.0,['ATRA'],['Atara Biotherapeutics'],['allogeneic Epstein Barr virus specific cytot...,"['EBV-induced Lymphomas', 'EBV-associated Mali...",NaN,NaN,0.336980,0.663020,1


In [460]:
df_temp_check.drop_duplicates(subset=['clinical_id','version']).shape

NameError: name 'df_temp_check' is not defined

In [468]:
get_CTs_from_ES("NCT02978326")

,abstract,arm_group,authors,authors_merged,clinical_id,completion_date,completion_date_source,condition,condition_mesh_terms,countries,created_at,data_source,eligibility,enrollment,firstreceived_date,firstreceived_date_source,id_info,innoplexus_id,intervention,keyword,language,last_updated,last_updated_dev,location,number_of_arms,official_title,overall_official,overall_status,oversight_info,phase,primary_completion_date,primary_completion_date_source,primary_outcome,priority,public_title,recruitment_status,responsible_party,secondary_outcome,source,source_url,sponsors,start_date,start_date_source,start_year,study_design,study_type,verification_date,verification_date_source
0,"This is a multicenter, randomized, double-blin...","[{'arm_group_label': 'SAGE-217 dosing', 'descr...","[{'author_name': 'Christopher Silber', 'role':...","[{'role': 'Study Director', 'new_author_id': '...",NCT02978326,"{'type': 'Actual', 'date': '2018-12-11'}","{'type': 'Actual', 'date': 'December 11, 2018'}",[Postpartum Depression],"[Depression, Depressive Disorder, Depression, ...",[United States],1480550400,ClinicalTrials.gov,"{'healthy_volunteers': 'No', 'gender': 'Female...","{'number': '153', 'type': 'Actual'}",2016-11-28,"November 28, 2016","{'org_study_id': '217-PPD-201', 'nct_id': 'NCT...",008a8607b48ed8c8553ef57303fb741584200caf425a8d...,"[{'arm_group_type': 'Experimental', 'intervent...",[Postpartum Care],english,1549377339,1553491683,"[{'facility': {'address': {'state': 'Arizona',...",2,"A Multicenter, Randomized, Double-Blind, Paral...","[{'role': 'Study Director', 'last_name': 'Chri...",Completed,{'has_dmc': 'No'},Phase 3,"{'type': 'Actual', 'date': '2018-11-15'}","{'type': 'Actual', 'date': 'November 15, 2018'}","[{'time_frame': '14 days', 'measure': 'Effect ...",1,A Study to Evaluate SAGE-217 in Subjects With ...,Completed,{'responsible_party_type': 'Sponsor'},"[{'time_frame': '75 days', 'measure': 'Safety ...",Sage Therapeutics,https://clinicaltrials.gov/show/NCT02978326,{'lead_sponsor': [{'agency': 'Sage Therapeutic...,2016-12-01,December 2016,2016,"[{'type': 'allocation', 'value': 'Randomized'}...",Interventional,2019-01-01,January 2019


In [470]:
df_final[df_final['clinical_id']=="NCT02978326"]

,clinical_id,val,overall_status_lower,number_of_arms,num_of_countries,num_of_sites,number_of_collaborators,minimum_age_month,maximum_age_month,type_of_lead_sponsor_Hospital,type_of_lead_sponsor_Individual,type_of_lead_sponsor_Industry,type_of_lead_sponsor_NIH,type_of_lead_sponsor_Other,type_of_lead_sponsor_US_Fed,type_of_lead_sponsor_University,gender_proper_Both,gender_proper_Female,gender_proper_Male,duration_in_days,trials_in_2_years,enrollment_num,os_activenotrecruiting,os_completed,os_enrollingbyinvitation,os_notyetrecruiting,os_random,os_recruiting,os_suspended,os_terminated,os_withdrawn,randomized_no,randomized_unknown,randomized_yes,masking_openlabel,masking_singleblind,masking_doubleblind,endpt_classifictn_safety,endpt_classifictn_efficacy,endpt_classifictn_pharmacokinetics,endpt_classifictn_effectiveness,armgrpclass_experimental,armgrpclass_active,armgrpclass_Placebo,armgrpclass_NoIntervention,armgrpclass_Sham,number_of_authors,number_of_princi_invest,authorclass_princiinvest,authorclass_studydir,authorclass_studychair,authorclass_sponsorinvestigator,completiondateclass_actual_completion,completiondateclass_anticipated_completion,Combination_Therapy,Comparison_Therapy,Monotherapy,purpose_None,purpose_Other,purpose_Prevention,purpose_Research,purpose_Supportive_Care,purpose_Treatment,intervention_Crossover_Assignment,intervention_Factorial_Assignment,intervention_None,intervention_Parallel_Assignment,intervention_Sequential_Assignment,intervention_Single_Group_Assignment,trial_activity_duration,phase_1,phase_2,phase_3,phase_4,phase_Others,is_rare_disease,KOL,count_of_drugs_exp,drug_condition_history_phase1,drug_condition_history_phase2,drug_condition_history_phase3,drug_condition_history_phase4,sponsor_history_phase1,sponsor_history_phase2,sponsor_history_phase3,sponsor_history_phase4,cong_drugcond_no_of_congresses,approved_drug_count_by_sponsor,diff_in_completiondate,if_all_drugs_for_some_indication,if_any_drug_prior_approved,indication_approved_for_all_drugs,no_of_drugs_approved_for_indication,variations_in_completiondate_using_versions,polar_surface_area,hydrogen_bond_acceptor_count,hydrogen_bond_donor_count,violations_in_octet_rule_count,heavy_atoms_count,logp_value,rotatable_bond_count,aromatic_rings_count,molecular_weight,no_of_metabolites_incr,no_of_metabolites_decr,no_of_targets,no_of_pathways,no_of_manufacturer,pharmacological_other,pharmacological_activator,pharmacological_agonist,pharmacological_antagonist,pharmacological_antibody,pharmacological_binder,pharmacological_inhibitor,biological_enzyme,biological_gpcr,biological_kinase,biological_nr,biological_transporter,biological_unspecified,biological_IC,type_biotech,type_small_molecule,state_gas,state_liquid,state_solid,status_of_drug_marketed,status_of_drug_non_marketed,status_of_drug_other,DrugGroup__approved,DrugGroup__experimental,DrugGroup__illicit,DrugGroup__investigational,DrugGroup__nutraceutical,DrugGroup__vet_approved,DrugGroup__withdrawn,Medium_aerosol,Medium_capsule,Medium_injection,Medium_kit,Medium_lotion,Medium_other,Medium_patch,Medium_powder,Medium_solution,Medium_suspension,Medium_tablet,isTop20,TA_class_TA_cardiology,TA_class_TA_dermatology,TA_class_TA_endocrinology_metabolic,TA_class_TA_gastroenterology,TA_class_TA_genetic,TA_class_TA_hematology,TA_class_TA_immunology,TA_class_TA_infections,TA_class_TA_musculoskeletal,TA_class_TA_nephrology,TA_class_TA_neurology,TA_class_TA_obstetrics_gynaecology,TA_class_TA_oncology,TA_class_TA_ophthalmology,TA_class_TA_others,TA_class_TA_respiratory,TA_class_TA_urology,impact_factor,no_of_publications,no_of_grants,no_of_patents
